

Important note in the numerical implementation of the factorized waveform:
-  spinning component PN amplitude modes $f^{\rm S}_{lm}$ (defined in [this cell](#fspin)), there are terms of $\mathcal{O}(\frac{\chi_A}{\delta})$ that diverge in the equal mass limit, i.e, $\delta \rightarrow 0$. However, for all odd values of $m$, the Newtonian multipole coefficient $c_{l + \epsilon}$ behaves as $c_{l + \epsilon} \propto \delta[1 + \mathcal{O}(\delta)]$ and vanishes in the equal mass $\delta \rightarrow 0$ limit. Therefore, the product $c_{l+\epsilon}f^{\rm S}_{lm}$ converges in the equal mass limit. In order to ensure that we obtain the mathematically correct flux in the equal mass limit, we choose our tolerance as $10^{-14}$ for the limit. That is, for $\delta > 10^{-14}$, we calculate $c_{l+\epsilon}$ and $f^{\rm S}_{lm}$ normally. For $\delta < 10^{-14}$, we calculate $\lim_{\delta \rightarrow 0}\left(\frac{c_{l + \epsilon}}{\delta}\right)$ and $\lim_{\delta \rightarrow 0}\left(\delta f^{\rm NS}_{lm}\right)$ instead.

- The original implementation of SEOBNR approximants rely on if-else/switch-case statements to populate waveform mode structures. In our implementation, we list each $l,m$ mode explicitly to avoid if-else statements inside a for loop for smoother computation. Additionally, the prescription given in the [NRPy+ Piecewise Expressions Tutorial](https://nbviewer.org/github/zachetienne/nrpytutorial/blob/master/Tutorial-Min_Max_and_Piecewise_Expressions.ipynb) will be used to generate conditional expressions to avoid checking for the equal mass limit discussed above.

-  As per the discussion above Equation 21 in [DCCT2300060](https://dcc.ligo.org/public/0186/T2300060/002/SEOBNRv5HM.pdf), the SEOBNRv5HM waveform only uses the (2,2), (2,1), (3,3), (3,2), (4,4), (4,3), and (5,5) modes to build the waveform. However, the flux is computed by summing up the $l \leq 8$ mode amplitudes. In this notebook, we list all the waveform modes for completeness. The resulting code, however, is generated by building only relevant modes as sympy expressions. This allows only for relevant parts of the notebook to enter the final SEOBNRv5_NRPy_opt code, making our implementation less bulky. (Future approximants may add or build on additional modes. thus, this notebook facilitates extensibility to future approximants!)
\end{itemize}

In [1]:
import sys#Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "Radiation"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

flux_list = []
strain_list = []
init_list = []

# Step : The Factorized Flux $\mathcal{F}_{\phi}$

The factorized flux is given in Equation 16 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
\mathcal{F}_{\phi} = -\frac{\Omega}{8\pi}\sum_{l = 2}^{8} \sum_{m = 1}^{l} m^2 |h^{\rm F}_{lm}|^2
$$

Where, $\Omega$ is the orbital frequency, and $|h^{\rm F}_{lm}|$ is the amplitude of the factorized waveform defined in [this cell](#hf)

In [2]:
flux_list.append(
"""                factorized_flux += m * m * strain_amplitude**2
"""
)

<a id='hf'></a>

# Step : The Factorized Waveform $h^{\rm F}_{lm}$ \[Back to [top](#toc)\]
$$\label{hf}$$

The factorized inspiral waveform modes are expressed in Equation 25 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
h^{\rm F}_{lm} = h^{\rm N}_{lm}\hat{S}_{\rm eff}T_{lm}f_{lm}e^{i\delta_{lm}}
$$

Where, $h^{\rm N}_{lm}$ is the leading (Newtonian) order waveform defined in [this cell](#hnewtonian), $\hat{S}_{\rm eff}$ is the effective source term defined in [this cell](#source), $T_{lm}$ is the contribution to the waveform obtained by resumming tail contributions defined in [this cell](#tail), $f_{lm}$ is the PN contribution to the amplitude defined in [this cell](#pn), and $\delta_{lm}$ is the PN contribution to the phase.

We note, first that the phase information is only relevant for (2,2), (2,1), (3,3), (3,2), (4,4), (4,3), and (5,5) modes for the waveform. The remaining modes are only used to calculate the flux for which we only require the amplitude component i.e $|h^{\rm F}_{lm}|$.

In [3]:
strain_list.append(
"""                hlms[f"({l} , {m})"] = newtonian_strain * self.effective_source[(l + m) % 2] * tail_term * pn_contribution_f * self.deltalm[f"({l} , {m})"]
"""
)
flux_list.append(
"""                strain_amplitude = newtonian_strain_amplitude * self.effective_source[(l + m) % 2] * tail_term * pn_contribution_f
"""
)               

<a id='hnewtonian'></a>

# Step : The Newtonian Waveform $h^{\rm N}_{lm}$ \[Back to [top](#toc)\]
$$\label{hnewtonian}$$

The Newtonian component of the inspiral waveform modes are expressed in Equation 26 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
h^{\rm N}_{lm} = \nu n_{lm}c_{l + \epsilon_{lm}}v_{\phi}^{l + \epsilon_{lm}}Y_{l - \epsilon_{lm},-m}\left(\frac{\pi}{2},\phi\right)
$$

Where, the symmetric mass ratio $\nu$ is defined in [this cell], the $\nu$-independent coefficient $n_{lm}$ is defined in [this cell](#n), the $\nu$-dependent coefficient $c_{l + \epsilon_{lm}}$ is defined in [this cell](#c), the PN circular velocity $v_{\phi}$ is defined in [this cell](#vphi), and $Y$ represents the spherical harmonic function listed in [this cell](#y). The parity $\epsilon_{lm}$ of the mode is given by

$$
\epsilon_{lm} = 
\bigg\lbrace
    \begin{array}{lr}
        0, & l + m \textrm{ even}\\
        1, & l + m \textrm{ odd}
    \end{array}
$$

$\phi$ is the orbital phase, given as an input.

In [4]:
strain_list.append(
"""                newtonian_strain = self.nu * n * self.c[f"{l + (l + m) % 2}"] * (self.vphi ** (l + (l + m) % 2)) * self.Y[m][l - (l + m) % 2] * sp.exp( -sp.I * m * self.phi)
"""
)

flux_list.append(
"""                newtonian_strain_amplitude = self.nu * n_abs * self.c[f"{l + (l + m) % 2}"] * (self.vphi ** (l + (l + m) % 2)) * self.Y[m][l - (l + m) % 2]
"""
)

<a id='source'></a>

# Step : The Source Term $\hat{S}_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{source}$$

The tail component of the inspiral waveform modes are expressed in Equation 32 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
\hat{S}_{\rm eff} = 
\bigg\lbrace
    \begin{array}{lr}
        H_{\rm eff} & l + m \textrm{ even},\\
        v_{\Omega}p_{\phi} & l + m \textrm{ odd},
    \end{array}
$$

Where, $H_{\rm eff}$,the effective Hamiltonian, is defined in [this cell](#heff), and v_{\Omega} is defined in [this cell](#v). $p_{\phi}$ is the azimuthal momentum which is given as an input.

In [5]:
init_list.append(
"""        self.effective_source = [] 
        self.effective_source.append(Heff)
        self.effective_source.append(self.vomega * self.pphi)
"""
)

<a id='tail'></a>

# Step : The Tail Term $T_{lm}$ \[Back to [top](#toc)\]
$$\label{tail}$$

The tail component of the inspiral waveform modes are expressed in Equation 33 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
T_{lm} = \frac{\Gamma (l + 1 - 2i\hat{k})}{\Gamma(l + 1)}e^{\pi \hat{k}}e^{2i\hat{k}\ln(2m\Omega r_0)}
$$

Where, $\Gamma$ is the gamma function implemented in sympy and defined as in [this Wikipedia article](https://en.wikipedia.org/wiki/Gamma_function). wavenumber $\hat{k}$ is defined in [this cell](#khat), the PN frequency $\Omega$ is defined in [this cell](#omegapn), and the constant $r_0$ is defined as:

$$
r_0 = \frac{2}{\sqrt e}
$$


The above expression is optimized mathematically before coding to exploit the properties of the gamma function. In this case, we note the simplification:

$$
\Gamma(l + 1) = l! \quad \forall l \in \mathbb{N}.
$$

which does not require using a generalized gamma function call.

Additionally, the full gamma function is not required when calculating the amplitude. Particularly, cf the properties section of the same [Wikipedia article](https://en.wikipedia.org/wiki/Gamma_function#Properties) to find the following simplification

$$
|\Gamma(1 + l + bi)|^2 = \frac{\pi b}{\sinh \pi b} \prod_{j = 1}^{l} (j^2 + b^2) = \frac{2\pi b}{e^{\pi b}- e^{-\pi b}} \prod_{j = 1}^{l} (j^2 + b^2)
$$

Where, in the last step, $\sinh$ is defined explicitly as sympy's cse does not exploit the definition of the $\sinh$ function.

In [6]:
strain_list.append(
"""                tail_term = (
                       sp.gamma(l + 1 - 2 * sp.I * self.khat[m]) 
                       * sp.exp(self.khat[m] * (sp.pi + 2 * sp.I * sp.log(2 * m * self.Omega * self.r0)))
                       / sp.factorial(l)
                   )
"""
)

init_list.append("""
        self.Tlm = {}
        tlmprod_fac = 1
        for m in range(1,9):
            k = m*self.Omega
            hathatk = self.Hreal*k
            hathatksq4 = 4 * hathatk * hathatk
            hathatk4pi = 4 * sp.pi * hathatk
            tlmprod_fac = 1
            tlmprefac = sp.sqrt(hathatk4pi / (1 - sp.exp(-hathatk4pi)))
            for j in range(1,9):
                z2 = sp.factorial(j)
                tlmprod_fac*= hathatksq4 + j**2
                if m>j:
                    continue
                self.Tlm.update({f"({j} , {m})" : tlmprefac*sp.sqrt(tlmprod_fac)/z2})
"""
)

flux_list.append(
"""                tail_term = self.Tlm[f"({l} , {m})"]
"""
)

init_list.append(
"""        self.r0 = 2 / sp.sqrt(sp.E)
"""
)

# Step : The Newtonian Waveform Components

<a id='n'></a>

# Step : The coefficient $n_{lm}$ \[Back to [top](#toc)\]
$$\label{n}$$

The $\nu$-independent coefficient of the Newtonian waveform modes is expressed in Equation 28 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
{n}_{lm} = 
\bigg\lbrace
    \begin{array}{lr}
        \frac{8\pi(im)^l}{(2l + 1)!!}\sqrt{\frac{(l+1)(l+2)}{(l)(l-1)}} & l + m \textrm{ even},\\
        \frac{-16i\pi(im)^l}{(2l + 1)!!}\sqrt{\frac{(2l+1)(l+2)(l^2 - m^2)}{(2l-1)(l+1)(l)(l-1)}} & l + m \textrm{ odd},
    \end{array}
$$

In [7]:
strain_list.append(
"""                n = (
                    (1 - (l + m) % 2) * (
                        (sp.Rational(8 , sp.factorial2(2 * l + 1)))
                        * sp.pi 
                        * (sp.I * m)**l 
                        * sp.sqrt(
                            sp.Rational( (l + 1)*(l + 2) , l * (l - 1))
                        )
                    )   
                    + 
                    ((l + m) % 2 )* ( 
                        (sp.Rational(-16 , sp.factorial2(2 * l + 1)))
                        * sp.pi 
                        * (sp.I * m)**l 
                        * sp.sqrt(
                            sp.Rational((l**2 -  m**2) * (l + 2) * (2 * l + 1) , (2 * l - 1) * (l + 1) * l * (l - 1))
                        )
                    )
                    )
"""
)

flux_list.append(
"""                n_abs = (
                    (1 - (l + m) % 2) * (
                        (sp.Rational(8 , sp.factorial2(2 * l + 1)))
                        * sp.pi 
                        * (m)**l 
                        * sp.sqrt(
                            sp.Rational( (l + 1)*(l + 2) , l * (l - 1))
                        )
                    )   
                    + 
                    ((l + m) % 2 )* ( 
                        (sp.Rational(16 , sp.factorial2(2 * l + 1)))
                        * sp.pi 
                        * (m)**l 
                        * sp.sqrt(
                            sp.Rational((l**2 -  m**2) * (l + 2) * (2 * l + 1) , (2 * l - 1) * (l + 1) * l * (l - 1))
                        )
                    )
                    )
"""
)

<a id='c'></a>

# Step : The coefficient $c_{l+ \epsilon_{lm}}$ \[Back to [top](#toc)\]
$$\label{c}$$

The $\nu$-dependent coefficient of the Newtonian waveform modes is expressed in Equation 29 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
{c}_{k} = \left(\frac{1 - \sqrt{1 - 4\nu}}{2}\right)^{k-1} + (-1)^k\left(\frac{1 + \sqrt{1 - 4\nu}}{2}\right)^{k-1}
$$

Since $f$ modes are only provided up to $l = 5$, we note that the values of $k = l + \epsilon_{lm}$ where the equal mass limit must be evaluated are given in the below table.

|$(l,m)$|$\epsilon_{lm}$|$k = l + \epsilon_{lm}$|$\lim_{\delta \rightarrow 0} \tfrac{c_{k}}{\delta}$|
|-------|---------------|-----------------------|----------------------------------------------------|
|$(2,1)$|$1$|$3$|$-1$|
|$(3,3)$|$0$|$3$|$-1$|
|$(3,1)$|$0$|$3$|$-1$|
|$(4,3)$|$1$|$5$|$-\tfrac{1}{2}$|
|$(4,1)$|$1$|$5$|$-\tfrac{1}{2}$|
|$(5,5)$|$0$|$5$|$-\tfrac{1}{2}$|

Also, some simple algrebra shows that $\sqrt{1 - 4\nu} = \delta$ which can be substituted in the above equation.

In [8]:
init_list.append(
"""        self.c = {}
        for k in range(2,10):
            self.c[f"{k}"] = (m1/M)**(k - 1) + ((-1)**k)*((m2/M)**(k-1))
        self.c["3"] += -1 * self.eqcond
        self.c["5"] += sp.Rational(-1 , 2) * self.eqcond
"""
)

<a id='y'></a>

# Step : $Y_{lm}$ \[Back to [top](#toc)\]
$$\label{y}$$

The equatorial spherical harmonics $Y_{lm}(\frac{\pi}{2},\phi)$ are given by [this table](https://en.wikipedia.org/wiki/Table_of_spherical_harmonics)

In [9]:
init_list.append(
"""        self.Y = [
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [
                0,
                f2r(0.3454941494713355),
                0,
                f2r(0.3231801841141506),
                0,
                f2r(0.32028164857621516),
                0,
                f2r(0.31937046138540076),
                0,
            ],
            [
                0,
                0,
                f2r(0.3862742020231896),
                0,
                f2r(0.33452327177864466),
                0,
                f2r(0.32569524293385776),
                0,
                f2r(0.32254835519288305),
            ],
            [
                0,
                0,
                0,
                f2r(0.4172238236327842),
                0,
                f2r(0.34594371914684025),
                0,
                f2r(0.331899519333737),
                0,
            ],
            [
                0,
                0,
                0,
                0,
                f2r(0.4425326924449826),
                0,
                f2r(0.3567812628539981),
                0,
                f2r(0.3382915688890245),
            ],
            [
                0,
                0,
                0,
                0,
                0,
                f2r(0.46413220344085826),
                0,
                f2r(0.3669287245764378),
                0,
            ],
            [0, 0, 0, 0, 0, 0, f2r(0.48308411358006625), 0, f2r(0.3764161087284946)],
            [0, 0, 0, 0, 0, 0, 0, f2r(0.5000395635705508), 0],
            [0, 0, 0, 0, 0, 0, 0, 0, f2r(0.5154289843972844)],
        ]
"""
)

<a id='pn'></a>

# Step : The PN amplitude component $f_{lm}$ \[Back to [top](#toc)\]
$$\label{pn}$$

The tail component of the inspiral waveform modes is expressed in Equation 34 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
f_{lm} = 
\bigg\lbrace
    \begin{array}{lr}
        (\rho_{lm})^l & m \textrm{ even}\\
        (\rho^{\rm NS}_{lm})^l + f^{\rm S}_{lm} & m \textrm{ odd}
    \end{array}     
$$

Where, $\rho_{lm}$ and its non-spinning component $\rho^{\rm NS}_{lm}$ are defined in [this cell](#rho), and the PN frequency $\Omega$ is defined in [this cell](#omegapn), and the odd-$m$ spinning component $f^{\rm S}_{lm}$ is defined in [this cell](#fspin). We also note that, as per Appendix B of [PB2023](https://arxiv.org/pdf/2303.18039.pdf), that $f$ modes are non-zero only up to $l = 5$, where only the $m = 5$ mode is non-zero.

Finally, it is important to discuss that, in the limit $\delta \rightarrow 0,\text{ } \delta \equiv \tfrac{m_1 - m_2}{M}$, only the non-vanishing term must be applied. That is, 

$$
f_{lm} = 
\bigg\lbrace
    \begin{array}{lr}
        (\rho_{lm})^l & m \textrm{ even}\\
        (\rho^{\rm NS}_{lm})^l + f^{\rm S}_{lm} & m \textrm{ odd, and } \delta \gt 10^{-14}\\
        \lim_{\delta \rightarrow 0} (\delta f^{\rm S}_{lm}) & m \textrm{ odd, and } \delta \lt 10^{-14}
    \end{array}     
$$

NOTE: Should define $1/\delta$ to avoid divergence in the limiting case.

In [10]:
strain_list.append(
"""                pn_contribution_f = self.noneqcond * self.rho[f"({l} , {m})"]**l
                if (m) % 2:
                    pn_contribution_f += self.noneqcond * self.fspin[f"({l} , {m})"] + self.eqcond * self.fspin_limit[f"({l} , {m})"]
                if l == 3 and m == 3:
                    pn_contribution_f += self.noneqcond * self.fspinimag + self.eqcond * self.fspinimag_limit
"""
)

flux_list.append(
"""                pn_contribution_f = self.noneqcond * self.rho[f"({l} , {m})"]**l
                if (m) % 2:
                    if (l < 5) or (l == 5 and m == 5):
                        pn_contribution_f += self.noneqcond * self.fspin[f"({l} , {m})"] + self.eqcond * self.fspin_limit[f"({l} , {m})"]
"""
)

# Step : PN Waveform Modes

<a id='rho'></a>

# Step : $\rho_{lm}$ \[Back to [top](#toc)\]
$$\label{rho}$$

$\rho_{lm}$ is defined in Appendix B of [PB2023](https://arxiv.org/pdf/2303.18039.pdf).

We begin with the (2,2) mode given in Equation B1a:

\begin{align*}
\rho_{22} &= 1
+ v_\Omega^2 \left(\tfrac{55}{84}\nu-\tfrac{43}{42}\right)
+ v_\Omega^3 \left[\left(\tfrac{2}{3}\nu-\tfrac{2}{3}\right) \chi _S-\tfrac{2}{3}\delta\chi _A\right]
+ v_\Omega^4 \Big[
\tfrac{19583}{42336} \nu^2-\tfrac{33025}{21168}\nu-\tfrac{20555}{10584}
+ \left(\tfrac{1}{2}-2 \nu \right) \chi _A^2+\delta  \chi _A \chi _S+\tfrac{1}{2}\chi _S^2\Big] \nonumber\\
&\quad
+ v_\Omega^5 \left[\delta  \left(-\tfrac{19}{42}\nu-\tfrac{34}{21}\right) \chi _A+\left(\tfrac{209}{126}\nu^2+\tfrac{49}{18}\nu-\tfrac{34}{21}\right) \chi _S\right] \nonumber \\
&\quad
+ v_\Omega^6 \Big[
\tfrac{10620745 \nu^3}{39118464}-\tfrac{6292061 \nu^2}{3259872}+\tfrac{41 \pi^2 \nu}{192}-\tfrac{48993925 \nu}{9779616}-\tfrac{428}{105}\text{eulerlog}(2,v_\Omega)+\tfrac{1556919113}{122245200} \nonumber \\
&\qquad 
 + \delta  \left(\tfrac{89}{126}-\tfrac{781}{252}\nu\right) \chi _A \chi _S
+\left(-\tfrac{27}{14}\nu^2-\tfrac{457}{504}\nu+\tfrac{89}{252}\right) \chi _A^2
+\left(\tfrac{10}{9}\nu^2 -\tfrac{1817}{504}\nu +\tfrac{89}{252}\right) \chi _S^2 \Big] \nonumber\\
&\quad
+ v_\Omega^7 \Big[
\delta  \left(\tfrac{97865}{63504}\nu^2 +\tfrac{50140}{3969}\nu +\tfrac{18733}{15876}\right) \chi _A
+\left(\tfrac{50803}{63504}\nu^3 -\tfrac{245717}{63504}\nu^2 +\tfrac{74749 }{5292}\nu +\tfrac{18733}{15876}\right) \chi _S
{+ \delta \chi _A^3 \left(\tfrac{1}{3}-\tfrac{4}{3}\nu\right)+\delta  (2 \nu +1) \chi _A \chi _S^2} \nonumber\\
&\qquad
{+ \left(-4 \nu ^2-3 \nu +1\right) \chi _A^2 \chi _S+\left(\nu +\tfrac{1}{3}\right) \chi _S^3}\Big] 
+ v_\Omega^8 \left[\tfrac{9202}{2205}\text{eulerlog}(2,v_\Omega)-\tfrac{387216563023}{160190110080}\right]\nonumber \\
&\quad 
+ v_\Omega^{10} \left[\tfrac{439877}{55566}\text {eulerlog}(2,v_\Omega)-\tfrac{16094530514677}{533967033600}\right], \\
\end{align*}

In addition, the following 2GSF corrections are added

$$
\Delta \rho_{22}^{(1)} = 21.2v_{\Omega}^{8} - 411v_{\Omega}^{10}
$$

In [11]:
init_list.append(
"""        self.rho["(2 , 2)"] = (1 +
            self.vomega**2 * (
                sp.Rational(55, 84) * self.nu
                - sp.Rational(43, 42)
            ) +
            self.vomega**3 * (
                (-2.0 * (self.chi_S + self.chi_A * self.delta - self.chi_S * self.nu)) / 3.0
            ) +
            self.vomega**4 * (
                sp.Rational(19583, 42336) * self.nu**2
                - sp.Rational(33025, 21168) * self.nu
                - sp.Rational(20555, 10584)
                + (sp.Rational(1, 2) - 2 * self.nu) * self.chi_A**2
                + self.delta * self.chi_A * self.chi_S
                + sp.Rational(1, 2) * self.chi_S**2
            ) +
            self.vomega**5 * (
                self.delta * (
                    -sp.Rational(19, 42) * self.nu
                    - sp.Rational(34, 21)
                ) * self.chi_A +
                (
                    sp.Rational(209, 126) * self.nu**2
                    + sp.Rational(49, 18) * self.nu
                    - sp.Rational(34, 21)
                ) * self.chi_S
            ) +
            self.vomega**6 * (
                sp.Rational(10620745, 39118464) * self.nu**3
                - sp.Rational(6292061, 3259872) * self.nu**2
                + sp.Rational(41, 192) * sp.pi**2 * self.nu
                - sp.Rational(48993925, 9779616) * self.nu
                - sp.Rational(428, 105) * eulerlog[2]
                + sp.Rational(1556919113, 122245200)
                + self.delta * (
                    sp.Rational(89, 126)
                    - sp.Rational(781, 252) * self.nu
                ) * self.chi_A * self.chi_S
                + (
                    -sp.Rational(27, 14) * self.nu**2
                    - sp.Rational(457, 504) * self.nu
                    + sp.Rational(89, 252)
                ) * self.chi_A**2
                + (
                    sp.Rational(10, 9) * self.nu**2
                    - sp.Rational(1817, 504) * self.nu
                    + sp.Rational(89, 252)
                ) * self.chi_S**2
            ) +
            self.vomega**7 * (
                self.delta * (
                    sp.Rational(97865, 63504) * self.nu**2
                    + sp.Rational(50140, 3969) * self.nu
                    + sp.Rational(18733, 15876)
                ) * self.chi_A
                + (
                    sp.Rational(50803, 63504) * self.nu**3
                    - sp.Rational(245717, 63504) * self.nu**2
                    + sp.Rational(74749, 5292) * self.nu
                    + sp.Rational(18733, 15876)
                ) * self.chi_S
                + self.delta * self.chi_A**3 * (
                    sp.Rational(1, 3)
                    - sp.Rational(4, 3) * self.nu
                )
                + self.delta * (2 * self.nu + 1) * self.chi_A * self.chi_S**2
                + (
                    sp.Rational(-4, 1) * self.nu**2
                    - sp.Rational(3, 1) * self.nu
                    + sp.Rational(1, 1)
                ) * self.chi_A**2 * self.chi_S
                + (self.nu + sp.Rational(1, 3)) * self.chi_S**3
            )
            + self.vomega**8 * (
                sp.Rational(9202, 2205) * eulerlog[2]
                - sp.Rational(387216563023, 160190110080)
            )
            + self.vomega**10 * (
                sp.Rational(439877, 55566) * eulerlog[2]
                - sp.Rational(16094530514677, 533967033600)
            )
            + self.nu * (
                21.2 * self.vomega**8 +
                -411 * self.vomega**10
            )
            )
"""
)

The (2,1) mode is given in Equation B2a:

\begin{align*}
\rho_{21}^\text{NS} &=1
+v_\Omega^2 \left(\tfrac{23}{84}\nu -\tfrac{59}{56}\right) 
+v_\Omega^4 \left(\tfrac{617}{4704}\nu^2 -\tfrac{10993}{14112}\nu -\tfrac{47009}{56448}\right)  +v_\Omega^6 \left[\tfrac{7613184941}{2607897600}-\tfrac{107}{105} \text { eulerlog }\left(1, v_\Omega\right)\right]  \nonumber \\
&\quad+ v_\Omega^8\left[-\tfrac{1168617463883}{911303737344}+\tfrac{6313}{5880}\text { eulerlog }\left(1, v_\Omega\right)\right]
+v_\Omega^{10}\left[-\tfrac{63735873771463}{16569158860800} + \tfrac{5029963}{5927040} \text{ eulerlog }\left(1, v_\Omega\right)\right] ,
\end{align*}

In addition, the following 2GSF corrections are added:

$$
\Delta\rho_{21}^{(1)} = 1.65v_{\Omega}^{6} + 26.5v_{\Omega}^{8} + 80v_{\Omega}^{10}
$$

In [12]:
init_list.append(
"""        self.rho["(2 , 1)"] = (
            1 + 
            self.vomega**2 * (sp.Rational(23, 84) * self.nu - sp.Rational(59, 56)) + 
            self.vomega**4 * (sp.Rational(617, 4704) * self.nu**2 - sp.Rational(10993, 14112) * self.nu - sp.Rational(47009, 56448)) + 
            self.vomega**6 * (sp.Rational(7613184941, 2607897600) - sp.Rational(107, 105) * eulerlog[1]) + 
            self.vomega**8 * (-sp.Rational(1168617463883, 911303737344) + sp.Rational(6313, 5880) * eulerlog[1]) + 
            self.vomega**10 * (-sp.Rational(63735873771463, 16569158860800) + sp.Rational(5029963, 5927040) * eulerlog[1]) +
            self.nu * (
                f2r(1.65) * self.vomega**6 +
                f2r(26.5) * self.vomega**8 +
                80 *self.vomega**10
            )
            )
"""
)

The (3,3) mode is given in Equation B3a:

\begin{align*}
\rho_{33}^\text{NS}&= 1+v_\Omega^2 \left(\tfrac{2}{3}\nu-\tfrac{7}{6}\right)  + v_\Omega^4\left(-\tfrac{6719}{3960}-\tfrac{1861}{990}\nu+\tfrac{149 }{330}\nu v_\Omega^2\right) \nonumber \\
&\quad
+v_\Omega^6 \left[\tfrac{3203101567}{227026800}+\left(-\tfrac{129509}{25740}+\tfrac{41 \pi^2}{192}\right) \nu-\tfrac{274621}{154440} \nu^2+\tfrac{12011 }{46332}\nu^3-\tfrac{26}{7} \text{eulerlog}\left(3, v_\Omega\right)\right]  \nonumber \\
&\quad
+v_\Omega^8\left[-\tfrac{57566572157}{8562153600}+\tfrac{13}{3} \text { eulerlog }\left(3, v_\Omega\right)\right] 
+v_\Omega^{10}\left[-\tfrac{903823148417327}{30566888352000}+\tfrac{87347}{13860}\text{eulerlog}\left(3, v_\Omega\right)\right],
\end{align*}

In addition, the following 2GSF terms are added:

$$
\Delta\rho_{33}^{(1)} = 12v_{\Omega}^{8} - 215v_{\Omega}^{10}
$$

In [13]:
init_list.append(
"""        self.rho["(3 , 3)"] = (
            1 + 
            self.vomega**2 * (
                sp.Rational(2, 3) * self.nu 
                - sp.Rational(7, 6)
            ) + 
            self.vomega**4 * (
                -sp.Rational(6719, 3960) 
                - sp.Rational(1861, 990) * self.nu 
                + sp.Rational(149, 330) * self.nu**2
            ) + 
            self.vomega**6 * (sp.Rational(3203101567, 227026800) + (-sp.Rational(129509, 25740) + sp.Rational(41, 192) * sp.pi**2) * self.nu -
                        sp.Rational(274621, 154440) * self.nu**2 + sp.Rational(12011, 46332) * self.nu**3 - sp.Rational(26, 7) * eulerlog[3]) + 
            self.vomega**8 * (-sp.Rational(57566572157, 8562153600) + sp.Rational(13, 3) * eulerlog[3]) + 
            self.vomega**10 * (-sp.Rational(903823148417327, 30566888352000) + sp.Rational(87347, 13860) * eulerlog[3]) + 
            self.nu * (
                12 * self.vomega**8 +
                -215 * self.vomega**10
                )
            )
"""
)

The (3,2) mode is given in Equation B6a:

\begin{align*}
\rho_{32} &= 1 + v_\Omega \frac{4 \nu \chi _S}{3 (1-3 \nu )}
+ v_\Omega^2 \left[
\frac{-\tfrac{32}{27}\nu ^2+\tfrac{223}{54}\nu-\tfrac{164}{135}}{1-3 \nu }
{-\frac{16 \nu ^2 \chi _S^2}{9 (1-3 \nu )^2}}
\right] \nonumber\\
&\quad
+ v_\Omega^3 \left[{\left(\tfrac{13 }{9}\nu +\tfrac{2}{9}\right) \frac{\delta \chi _A}{1-3 \nu }
+\left(\tfrac{607}{81}\nu ^3 +\tfrac{503}{81}\nu ^2 -\tfrac{1478}{405}\nu +\tfrac{2}{9}\right)  \frac{\chi _S}{(1-3 \nu )^2} } 
{+\frac{320 \nu ^3 \chi _S^3}{81 (1-3 \nu )^3}}\right]\nonumber\\
&\quad
+ v_\Omega^4 \Bigg[
\frac{\tfrac{77141}{40095} \nu ^4 -\tfrac{508474 }{40095}\nu ^3 -\tfrac{945121 }{320760}\nu ^2 +\tfrac{1610009 \nu }{320760}-\tfrac{180566}{200475}}{(1-3 \nu )^2}
+ {\left(4 \nu ^2-3 \nu +\tfrac{1}{3}\right) \frac{\chi _A^2}{1-3 \nu }
+ \left(-\tfrac{50}{27}\nu ^2-\tfrac{88}{27}\nu+\tfrac{2}{3}\right) \frac{\delta \chi _A \chi _S}{(1-3 \nu )^2} } \nonumber\\
&\qquad
{
+ \left(-\tfrac{2452}{243} \nu ^4 -\tfrac{1997}{243} \nu ^3 +\tfrac{1435}{243}\nu ^2 -\tfrac{43}{27}\nu +\tfrac{1}{3}\right)  \frac{\chi _S^2}{(1-3 \nu )^3}}
\Bigg] \nonumber\\
&\quad
+ {v_\Omega^5 \Bigg[
\left(-\tfrac{1184225 }{96228}\nu ^5 -\tfrac{40204523}{962280} \nu ^4 +\tfrac{101706029 }{962280}\nu ^3 -\tfrac{14103833 }{192456}\nu ^2 +\tfrac{20471053}{962280}\nu -\tfrac{2788}{1215}\right)  \frac{\chi _S}{(1-3 \nu )^3}
+ \left(\tfrac{608}{81} \nu ^3+\tfrac{736}{81} \nu ^2 -\tfrac{16  }{9}\nu\right)  \frac{\delta \chi _A \chi _S^2}{(1-3 \nu )^3}} \nonumber\\
&\qquad
{+\left(\tfrac{889673}{106920}\nu ^3-\tfrac{75737}{5346} \nu ^2+\tfrac{376177 }{35640}\nu -\tfrac{2788}{1215}\right)  \frac{\delta  \chi _A}{(1-3 \nu )^2}
+\left(\tfrac{96176 }{2187}\nu ^5 +\tfrac{43528}{2187} \nu ^4-\tfrac{40232 }{2187}\nu ^3 +\tfrac{376 }{81}\nu ^2 -\tfrac{8 \nu }{9}\right)  \frac{\chi _S^3}{(1-3 \nu )^4}} \nonumber\\
&\qquad
{+\left(-\tfrac{32 }{3}\nu ^3+8 \nu ^2-\tfrac{8}{9}\nu\right)  \frac{\chi _A^2 \chi _S}{(1-3 \nu )^2} \Bigg]}
+v_\Omega^6\left[\tfrac{5849948554}{940355325}-\tfrac{104 \text { eulerlog}(2,v_\Omega)}{63}\right] 
+v_\Omega^8 \left[\tfrac{17056 \text { eulerlog}(2,v_\Omega)}{8505}-\tfrac{10607269449358}{3072140846775}\right] \nonumber \\
&\qquad
+ {v_\Omega^{10}\left[- \tfrac{1312549797426453052}{176264081083715625} +\tfrac{18778864 \text { eulerlog}(2,v_{\Omega} )}{12629925}   \right]},
\end{align*}

In addition, the following 2GSF term is added:

$$
\Delta\rho_{32}^{(1)} = 0.333*v_{\Omega}^{6} - 6.5v_{\Omega}^{8} + 80v_{\Omega}^{10}
$$

In [14]:
init_list.append(
"""        self.rho["(3 , 2)"] = (
            1 
            + self.vomega * (
                (4 * self.nu * self.chi_S) / (3 * (1 - 3 * self.nu))
            )
            + self.vomega**2 * (
                ( sp.Rational(-32, 27) * self.nu**2 + sp.Rational(223, 54) * self.nu - sp.Rational(164, 135)) / (1 - 3 * self.nu) 
                -(16 * self.nu**2 * self.chi_S**2) / (9 * (1 - 3 * self.nu)**2)
            ) 
            + self.vomega**3 * (
                (sp.Rational(13, 9) * self.nu + sp.Rational(2, 9)) * (self.delta * self.chi_A) / (1 - 3 * self.nu) 
                + (sp.Rational(607, 81) * self.nu**3 + sp.Rational(503, 81) * self.nu**2 - sp.Rational(1478, 405) * self.nu + sp.Rational(2, 9))* self.chi_S / (1 - 3 * self.nu)**2 
                + (320 * self.nu**3 * self.chi_S**3) / (81 * (1 - 3 * self.nu)**3)
            )
            + self.vomega**4 * (
                (sp.Rational(77141, 40095) * self.nu**4 - sp.Rational(508474, 40095) * self.nu**3 
                 - sp.Rational(945121, 320760) * self.nu**2 + sp.Rational(1610009, 320760) * self.nu - sp.Rational(180566, 200475)
                ) / (1 - 3 * self.nu)**2 
                +(4 * self.nu**2 - 3 * self.nu + sp.Rational(1, 3)) * (self.chi_A**2) / (1 - 3 * self.nu)
                +(sp.Rational(-50, 27) * self.nu**2 - sp.Rational(88, 27) * self.nu + sp.Rational(2, 3)) * (self.delta * self.chi_A * self.chi_S) / (1 - 3 * self.nu)**2
                +(sp.Rational(-2452, 243) * self.nu**4 - sp.Rational(1997, 243) * self.nu**3 
                  + sp.Rational(1435, 243) * self.nu**2 - sp.Rational(43, 27) * self.nu + sp.Rational(1, 3)
                 ) * (self.chi_S**2) / ((1 - 3 * self.nu)**3)
            ) 
            + self.vomega**5 * (
                (sp.Rational(-1184225, 96228) * self.nu**5 - sp.Rational(40204523, 962280) * self.nu**4 
                 + sp.Rational(101706029, 962280) * self.nu**3 - sp.Rational(14103833, 192456) * self.nu**2 
                 + sp.Rational(20471053, 962280) * self.nu - sp.Rational(2788, 1215)
                ) * self.chi_S / (1 - 3 * self.nu)**3 
                + (sp.Rational(608, 81) * self.nu**3 + sp.Rational(736, 81) * self.nu**2 
                  - sp.Rational(16, 9) * self.nu
                 ) * (self.delta * self.chi_A * self.chi_S**2) / (1 - 3 * self.nu)**3 
                + (sp.Rational(889673, 106920) * self.nu**3 - sp.Rational(75737, 5346) * self.nu**2 
                   + sp.Rational(376177, 35640) * self.nu - sp.Rational(2788, 1215)
                  ) * (self.delta * self.chi_A) / (1 - 3 * self.nu)**2 
                + (sp.Rational(96176, 2187) * self.nu**5 + sp.Rational(43528, 2187) * self.nu**4 
                   - sp.Rational(40232, 2187) * self.nu**3 + sp.Rational(376, 81) * self.nu**2 - sp.Rational(8, 9) * self.nu
                  ) * (self.chi_S**3) / (1 - 3 * self.nu)**4 
                + (sp.Rational(-32, 3) * self.nu**3 + 8 * self.nu**2 - sp.Rational(8, 9) * self.nu
                  ) * (self.chi_A**2 * self.chi_S) / ((1 - 3 * self.nu)**2)
            ) 
            + self.vomega**6 * ( 
                sp.Rational(5849948554, 940355325) - sp.Rational(104, 63) * eulerlog[2]
            ) 
            +  self.vomega**8 * (
                sp.Rational(17056, 8505) * eulerlog[2] - sp.Rational(10607269449358, 3072140846775)
            ) 
            + self.vomega**10 * (
                -sp.Rational(1312549797426453052, 176264081083715625) + sp.Rational(18778864, 12629925) * eulerlog[2]
            )
            + self.nu*(
                + f2r(.333) * self.vomega**6
                - f2r(6.5) * self.vomega**8
                + 98 * self.vomega**10
            )
            )
"""
)

The (3,1) mode is given in Equation B8:

\begin{align*}
\rho_{31}^\text{NS} &= 1-v_\Omega^2\left[\frac{2}{9}\nu+\frac{13}{18}\right] 
+ v_\Omega^4  \left[-\frac{829}{1782}\nu^2-\frac{1685}{1782}\nu+\frac{101}{7128}\right] 
+ v_\Omega^6 \left[\frac{11706720301}{6129723600}-\frac{26}{63}\text{eulerlog}(1,v_\Omega^2)\right] \nonumber\\
&\quad
+ v_\Omega^8 \left[\frac{169}{567}\text{eulerlog}(1,v_\Omega^2) +\frac{2606097992581}{4854741091200}\right],
\end{align*}

Here, we document pyseobnr's use of $\text{eulerlog}(1,v_\Omega)$ instead of $\text{eulerlog}(1,v_\Omega^2)$

In [15]:
init_list.append(
"""        self.rho["(3 , 1)"] = (
            1
            - self.vomega**2 * (
                sp.Rational(2, 9) * self.nu
                + sp.Rational(13, 18)
            )
            + self.vomega**4 * (
                -sp.Rational(829, 1782) * self.nu**2
                - sp.Rational(1685, 1782) * self.nu
                + sp.Rational(101, 7128)
            )
            + self.vomega**6 * (
                sp.Rational(11706720301, 6129723600)
                - sp.Rational(26, 63) * eulerlog[1]
            )
            + self.vomega**8 * (
                sp.Rational(169, 567) * eulerlog[1] 
                + sp.Rational(2606097992581, 4854741091200)
            )
        
        )       
"""
)

The (4,4) mode is given in Equation B:

\begin{align*}
\rho_{44} &= 1+v_\Omega^2\left[\tfrac{1614-5870 \nu +2625 \nu ^2}{1320 (-1+3 \nu )}\right] + v_\Omega^3 \left[\left(\tfrac{2}{3}-\tfrac{41 \nu }{15}+\tfrac{14 \nu ^2}{5}\right) \tfrac{1}{(-1+3 \nu )}\chi_S 
+\delta \left(\tfrac{2}{3}-\tfrac{13 \nu }{5}\right) \tfrac{1}{(-1+3 \nu )}\chi _A\right]  \nonumber \\
&\quad
+v_\Omega^4\left[-\tfrac{14210377}{8808800 (1-3 \nu )^2}+\tfrac{32485357 \nu
}{4404400 (1-3 \nu )^2}-\tfrac{1401149 \nu ^2}{1415700 (1-3 \nu )^2} 
-\tfrac{801565 \nu ^3}{37752 (1-3 \nu )^2}+\tfrac{3976393 \nu ^4}{1006720 (1-3 \nu )^2}+\tfrac{1}{2}\chi _A^2-2 \nu  \chi _A^2+\delta \chi _A \chi _S+\tfrac{1}{2}\chi _S^2\right]  \nonumber \\
&\quad+ v_\Omega^5\Bigg[\left(-\tfrac{69}{55}+\tfrac{16571 \nu }{1650}-\tfrac{2673 \nu ^2}{100}+\tfrac{8539 \nu ^3}{440}+\tfrac{591 \nu ^4}{44}\right) \tfrac{1}{(1-3 \nu)^2}\chi _S
+\delta
\left(-\tfrac{69}{55}+\tfrac{10679 \nu }{1650}-\tfrac{1933 \nu ^2}{220}+\tfrac{597 \nu ^3}{440}\right) \tfrac{1}{ (1-3 \nu )^2}\chi_A\Bigg]  \nonumber \\
&\quad+v_\Omega^6 \left[\tfrac{16600939332793}{1098809712000}-\tfrac{12568 }{3465}\text{eulerlog}\left(4,v_\Omega\right)\right]  +v_\Omega^8\left[-\tfrac{172066910136202271}{19426955708160000}+\tfrac{845198 }{190575}\text{eulerlog}\left(4,v_\Omega\right)\right] \nonumber \\
&\quad+ v_\Omega^{10}\left[-\tfrac{17154485653213713419357}{568432724020761600000}+\tfrac{22324502267}{3815311500} \text{eulerlog}\left(4,v_\Omega\right)\right]\,,
\end{align*}

In addition, the following 2GSF correction is added:

$$
\Delta \rho_{44}^{(1)} = -3.56v_{\Omega}^{6} + 15.6v_{\Omega}^{8} - 216v_{\Omega}^{10}
$$

In [16]:
init_list.append(
"""        self.rho["(4 , 4)"] = (
            1 
            + self.vomega**2 * (
                (sp.Rational(1614 , 1320) 
                 - sp.Rational(5870 , 1320) * self.nu 
                 + sp.Rational(2625 , 1320) * self.nu**2
                ) / (-1 + 3 * self.nu)
            ) 
            + self.vomega**3 * (
                self.chi_A * (sp.Rational(2 , 3) - sp.Rational(13 , 5) * self.nu) * self.delta 
                + self.chi_S * (sp.Rational(2 , 3) - sp.Rational(41 , 15) * self.nu + sp.Rational(14 , 5) * self.nu**2)
            ) 
            / (-1 + 3 * self.nu)
            + self.vomega**4 * (
                -sp.Rational(14210377, 8808800) / (1 - 3 * self.nu)**2 +
                sp.Rational(32485357, 4404400) * self.nu / (1 - 3 * self.nu)**2 -
                sp.Rational(1401149, 1415700) * self.nu**2 / (1 - 3 * self.nu)**2 -
                sp.Rational(801565, 37752) * self.nu**3 / (1 - 3 * self.nu)**2 +
                sp.Rational(3976393, 1006720) * self.nu**4 / (1 - 3 * self.nu)**2 +
                sp.Rational(1, 2) * self.chi_A**2 - 2 * self.nu * self.chi_A**2 +
                self.delta * self.chi_A * self.chi_S + sp.Rational(1, 2) * self.chi_S**2
            ) 
            + self.vomega**5 * (
                ( -sp.Rational(69, 55) + sp.Rational(16571, 1650) * self.nu - 
                  sp.Rational(2673, 100) * self.nu**2 +
                  sp.Rational(8539, 440) * self.nu**3 +
                  sp.Rational(591, 44) * self.nu**4 ) *
                (1 / (1 - 3 * self.nu)**2) * self.chi_S +
                self.delta * ( -sp.Rational(69, 55) + sp.Rational(10679, 1650) * self.nu - 
                               sp.Rational(1933, 220) * self.nu**2 +
                               sp.Rational(597, 440) * self.nu**3 ) * 
                self.chi_A / ((1 - 3 * self.nu)**2)
            ) 
            + self.vomega**6 * (
                sp.Rational(16600939332793, 1098809712000) - 
                sp.Rational(12568, 3465) * eulerlog[4]
            ) 
            + self.vomega**8 * (
                -sp.Rational(172066910136202271, 19426955708160000) + 
                sp.Rational(845198, 190575) * eulerlog[4]
            ) 
            + self.vomega**10 * (
                -sp.Rational(17154485653213713419357, 568432724020761600000) +
                sp.Rational(22324502267, 3815311500) * eulerlog[4]
            )
            + self.nu * (
                -f2r(3.56) * self.vomega**6 +
                f2r(15.6) * self.vomega**8 +
                -216 * self.vomega**10
            )
        )
"""
)

The (4,3) mode is given in Equation B:

\begin{align*}
\rho_{43}^\text{NS} &= 1 + \frac{v_\Omega^2}{1-2 \nu } \left(-\tfrac{10}{11} \nu ^2+\tfrac{547}{176}\nu-\tfrac{111}{88}\right) -\tfrac{6894273}{7047040} v_\Omega^4
+v_\Omega^6\left[\tfrac{1664224207351}{195343948800}-\tfrac{1571}{770} \text { eulerlog}(3,v_\Omega)\right] \nonumber \\
&\quad
+  v_\Omega^8\left[-\tfrac{2465107182496333}{460490801971200}  + \tfrac{174381}{67760} \text { eulerlog}(3,v_{\Omega} )   \right],
\end{align*}

In addition, the following 2GSF corrections are added:

$$
\Delta_{43}^{(1)} = -0.654*v_{\Omega}^{4} - 3.69*v_{\Omega}^{6} + 18.5*v_{\Omega}^{8}
$$

In [17]:
init_list.append(
"""        self.rho["(4 , 3)"] = (
            1 + 
            self.vomega**2 / (1 - 2 * self.nu) * (-sp.Rational(10, 11) * self.nu**2 + sp.Rational(547, 176) * self.nu - sp.Rational(111, 88)) - 
            sp.Rational(6894273, 7047040) * self.vomega**4 + 
            self.vomega**6 * (sp.Rational(1664224207351, 195343948800) - sp.Rational(1571, 770) * eulerlog[3]) + 
            self.vomega**8 * (-sp.Rational(2465107182496333, 460490801971200) + sp.Rational(174381, 67760) * eulerlog[3]) + 
            self.nu * (
                -f2r(0.654) * self.vomega**4 +
                -f2r(3.69) * self.vomega**6 +
                f2r(18.5) * self.vomega**8
            )
            )
"""
)

The (4,2) mode is given in Equation B:

\begin{align*}
\rho_{42}&=1+\frac{285\,\nu^2-3530\,\nu+1146}{1320\,(3\,\nu-1)} v_\Omega^2 - \frac{v_\Omega^3}{15(1-3\nu)} \left[(78\nu^2 - 59\nu + 10)\chi_S +(10-21\nu)\delta\,\chi_A \right] \nonumber \\
&\quad+\frac{-379526805\,\nu^4-3047981160\,\nu^3+1204388696\,\nu^2+295834536\,\nu-114859044}{317116800\,(1-3\,\nu)^2} v_\Omega^4 \nonumber\\
&\quad+ \left[\frac{848238724511}{219761942400}-\frac{3142}{3465}\text{eulerlog}(2,v_\Omega^2)\right] v_\Omega^6\,,
\end{align*}

Here we document pyseobnr's $\text{eulerlog}(2,v_\Omega)$ instead of $\text{eulerlog}(2,v_\Omega^2)$

In [18]:
init_list.append(
"""        self.rho["(4 , 2)"] = (
            1 
            + self.vomega **2 * ( 
                (sp.Rational(1146 , 1320) 
                 - sp.Rational(3530 , 1320) * self.nu 
                 + sp.Rational(285 , 1320) * self.nu**2
                ) / (-1 + 3*self.nu)
            )
            + self.vomega ** 3 * (
                (
                    self.chi_A * (sp.Rational(10 , 15) - sp.Rational(21 , 15) * self.nu) * self.delta 
                    + self.chi_S * (sp.Rational(10 , 15) - sp.Rational(59 , 15) * self.nu + sp.Rational(78 , 15) * self.nu**2)
                ) / (-1 + 3*self.nu)
            )
            + self.vomega ** 4 * (
                (
                    -sp.Rational(114859044 , 317116800)
                    + sp.Rational(295834536 , 317116800) * self.nu
                    + sp.Rational(1204388696 , 317116800) * self.nu**2
                    - sp.Rational(3047981160 , 317116800) * self.nu**3
                    - sp.Rational(379526805 , 317116800) * self.nu**4
                ) / ((-1 + 3*self.nu)**2)
            )
            + self.vomega ** 6 *(
                sp.Rational(848238724511 , 219761942400)
                -sp.Rational(3142 , 3465) * eulerlog[2]
            )
        )
"""
)

The (4,1) mode is given in Equation B:

\begin{align*}
\rho_{41}^\text{NS} &= 1 +\frac{288\,\nu^2-1385\,\nu+602}{528\,(2\,\nu-1)}v_\Omega^2
-\frac{7775491}{21141120} v_\Omega^4 + \left[\frac{1227423222031}{1758095539200}-\frac{1571}{6930}\text{eulerlog}(1,v_\Omega^2)\right]\, v_\Omega^6 \,
\end{align*}

Here, we instead document pyseobnr's use of $\text{eulerlog}(1,v_\Omega)$ instead of $\text{eulerlog}(1,v_\Omega^2)$

In [19]:
init_list.append(
"""        self.rho["(4 , 1)"] = (
            1 + 
            (
                (
                    sp.Rational(288 , 528) * self.nu**2 
                    - sp.Rational(1385 , 528) * self.nu 
                    + sp.Rational(602 , 528)
                ) / 
                (2 * self.nu - 1)
            ) * self.vomega**2 - 
            (
                sp.Rational(7775491, 21141120)
            ) * self.vomega**4 + 
            (
                sp.Rational(1227423222031, 1758095539200)
                - sp.Rational(1571, 6930) * eulerlog[1]
            ) * self.vomega**6
            )
"""   
)

The (5,5) mode is given in Equation B:

\begin{align*}
\rho_{55}^{\mathrm{NS}} =& 1+v_\Omega^2\left[\tfrac{487}{390 (-1+2 \nu )}-\tfrac{649 \nu }{195 (-1+2 \nu )}+\tfrac{256 \nu ^2}{195 (-1+2 \nu )}\right] -\tfrac{3353747}{2129400} v_\Omega^4 \nonumber \\
   &+v_\Omega^6\left[\tfrac{190606537999247}{11957879934000}-\tfrac{1546}{429}\text{eulerlog}\left(5,v_\Omega\right) \right] +v_\Omega^8 \left[-\tfrac{1213641959949291437}{118143853747920000}+\tfrac{376451 }{83655}\text{eulerlog}\left(5,v_\Omega\right)\right] \nonumber \\
   &+v_\Omega^{10}\left[-\tfrac{150082616449726042201261}{4837990810977324000000}+\tfrac{2592446431}{456756300}\text{eulerlog}\left(5,v_\Omega\right)\right]\,,
\end{align*}

In addition, the following 2GSF corrections are added:

$$
\Delta \rho_{55}^{(1)} = -2.61v_{\Omega}^{6} + 1.25v_{\Omega}^{4} - 35.7v_{\Omega}^{8}
$$

In [20]:
init_list.append(
"""        self.rho["(5 , 5)"] = (
            1 + 
            self.vomega**2 * (sp.Rational(487, 390) / (-1 + 2 * self.nu) - sp.Rational(649, 195) * self.nu / (-1 + 2 * self.nu) +
                         sp.Rational(256, 195) * self.nu**2 / (-1 + 2 * self.nu)) - 
            sp.Rational(3353747, 2129400) * self.vomega**4 + 
            (sp.Rational(190606537999247, 11957879934000) - sp.Rational(1546, 429) * eulerlog[5]) * self.vomega**6 + 
            (-sp.Rational(1213641959949291437, 118143853747920000) + sp.Rational(376451, 83655) * eulerlog[5]) * self.vomega**8 + 
            (-sp.Rational(150082616449726042201261, 4837990810977324000000) + sp.Rational(2592446431, 456756300) * eulerlog[5]) * self.vomega**10 +
            self.nu * (
                -f2r(2.61) * self.vomega**4 + 
                f2r(1.25) * self.vomega**6 +
                -f2r(35.7) * self.vomega**8
            )
            )
"""
)

The remaining $l = 5$ modes are given in Equation B:

\begin{align*}
\rho_{54}&=1+\frac{33320\,\nu^3-127610\,\nu^2+96019\,\nu-17448}{13650\,(5\,\nu^2-5\,\nu+1)} v_\Omega^2 -\frac{16213384}{15526875} v_\Omega^4 \,, \\
%%(5,3)
\rho^{\rm NS}_{53}&=1+\frac{176\,\nu^2-850\,\nu+375}{390\,(2\,\nu-1)} v_\Omega^2 -\frac{410833}{709800} v_\Omega^4\,, \\
%%(5,2)
\rho_{52}&=1+\frac{21980\,\nu^3-104930\,\nu^2+84679\,\nu-15828}{13650\,(5\,\nu^2-5\,\nu+1)} v_\Omega^2 -\frac{7187914}{15526875} v_\Omega^4 \,, \\
%%(5,1)
\rho^{\rm NS}_{51}&=1+\frac{8\,\nu^2-626\,\nu+319}{390\,(2\,\nu-1)}v_\Omega^2 -\frac{31877}{304200} v_\Omega^4\,, \\
\end{align*}

In [21]:
init_list.append(
"""        self.rho["(5 , 4)"] = (
            1 + 
            (
                (33320 * self.nu**3 - 127610 * self.nu**2 + 96019 * self.nu - 17448) /
                (13650 * (5 * self.nu**2 - 5 * self.nu + 1))
            ) * self.vomega**2 - 
            sp.Rational(16213384, 15526875) * self.vomega**4
            )
        
        self.rho["(5 , 3)"] = (
            1 + 
            (
            (176 * self.nu**2 - 850 * self.nu + 375) / (390 * (2 * self.nu - 1))
            ) * self.vomega**2 - 
            sp.Rational(410833, 709800) * self.vomega**4
            )
        
        self.rho["(5 , 2)"] = (
            1 + 
            (
                (21980 * self.nu**3 - 104930 * self.nu**2 + 84679 * self.nu - 15828) /
                (13650 * (5 * self.nu**2 - 5 * self.nu + 1))
            ) * self.vomega**2 - 
            sp.Rational(7187914, 15526875) * self.vomega**4
            )
        
        self.rho["(5 , 1)"] = (
            1 + 
            
            (8 * self.nu**2 - 626 * self.nu + 319) / (390 * (2 * self.nu - 1)) * self.vomega**2 - 
            sp.Rational(31877, 304200) * self.vomega**4
            )
"""
)

The $l = 6$ modes are given in Equation B:

\begin{align*}
\rho_{66} &= 1+\frac{273\,\nu^3-861\,\nu^2+602\,\nu-106}{84\,(5\,\nu^2-5\,\nu+1)} v_\Omega^2 - \frac{1025435}{659736} v_\Omega^4 \,, \\
%%(6,5)
\rho_{65}& = 1 + \frac{220\,\nu^3-910\,\nu^2+838\,\nu-185}{144\,(3\,\nu^2-4\,\nu+1)} v_\Omega^2\,, \\
%%(6,4)
\rho_{64} &= 1 + \frac{133\,\nu^3-581\,\nu^2+462\,\nu-86}{84\,(5\,\nu^2-5\,\nu+1)} v_\Omega^2 - \frac{476887}{659736} v_\Omega^4 \,, \\
%%(6,3)
\rho_{63} &= 1 + \frac{156\,\nu^3-750\,\nu^2+742\,\nu-169}{144\,(3\,\nu^2-4\,\nu+1)} v_\Omega^2\,, \\
%%(6,2)
\rho_{62} &= 1 + \frac{49\,\nu^3-413\,\nu^2+378\,\nu-74}{84\,(5\,\nu^2-5\,\nu+1)} v_\Omega^2 - \frac{817991}{3298680} v_\Omega^4 \,, \\
%%(6,1)
\rho_{61} &= 1 + \frac{124\,\nu^3-670\,\nu^2+694\,\nu-161}{144\,(3\,\nu^2-4\,\nu+1)} v_\Omega^2\,, \\
\end{align*}

In [22]:
init_list.append(
"""        self.rho["(6 , 6)"] = (
            1 + 
            (
                (273 * self.nu**3 - 861 * self.nu**2 + 602 * self.nu - 106) /
                (84 * (5 * self.nu**2 - 5 * self.nu + 1))
            ) * self.vomega**2 - 
            sp.Rational(1025435, 659736) * self.vomega**4
            )
        
        self.rho["(6 , 5)"] = (
            1 + 
            (
                (220 * self.nu**3 - 910 * self.nu**2 + 838 * self.nu - 185) /
                (144 * (3 * self.nu**2 - 4 * self.nu + 1))
            ) * self.vomega**2
            )
        
        self.rho["(6 , 4)"] = (
            1 + 
            self.vomega**2 * (
                (133 * self.nu**3 
                 - 581 * self.nu**2 
                 + 462 * self.nu 
                 - 86
                ) /
                (
                    84 * (
                        5 * self.nu**2 
                        - 5 * self.nu + 1
                    )
                )
            ) - 
            sp.Rational(476887, 659736) * self.vomega**4
            )
        
        self.rho["(6 , 3)"] = (
            1 + 
            (
                (156 * self.nu**3 - 750 * self.nu**2 + 742 * self.nu - 169) /
                (144 * (3 * self.nu**2 - 4 * self.nu + 1))
            ) * self.vomega**2
            )
        
        self.rho["(6 , 2)"] = (
            1 + 
            (
                (49 * self.nu**3 - 413 * self.nu**2 + 378 * self.nu - 74) /
                (84 * (5 * self.nu**2 - 5 * self.nu + 1))
            ) * self.vomega**2 - 
            sp.Rational(817991, 3298680) * self.vomega**4
            )
        
        self.rho["(6 , 1)"] = (
            1 + 
            (
                (124 * self.nu**3 - 670 * self.nu**2 + 694 * self.nu - 161) /
                (144 * (3 * self.nu**2 - 4 * self.nu + 1))
            ) * self.vomega**2
            )
"""
)

The $l = 7$ modes are given in Equation B:

\begin{align*}
%%(7,7)
\rho_{77} &= 1 + \frac{1380 \nu ^3-4963 \nu ^2+4246 \nu -906}{714 \left(3 \nu ^2-4 \nu +1\right)} v_{\Omega }^2 \,, \\
%%(7,6)
\rho_{76} &= 1 + \frac{6104 \nu ^4-29351 \nu ^3+37828 \nu^2-16185 \nu +2144} {1666 \left(7 \nu ^3-14 \nu ^2+7 \nu-1\right)} v_{\Omega }^2 \,, \\
%%(7,5)
\rho_{75} &= 1 + \frac{804 \nu ^3-3523 \nu ^2+3382 \nu -762}{714 \left(3 \nu ^2-4 \nu +1\right)} v_{\Omega }^2 \,, \\
%%(7,4)
\rho_{74} &= 1 + \frac{41076 \nu ^4-217959 \nu ^3+298872 \nu^2-131805 \nu +17756} {14994 \left(7 \nu ^3-14 \nu ^2+7 \nu-1\right)} v_{\Omega }^2 \,, \\
%%(7,3)
\rho_{73} &= 1 + \frac{420 \nu ^3-2563 \nu ^2+2806 \nu -666}{714 \left(3 \nu ^2-4 \nu +1\right)} v_{\Omega }^2 \,, \\
%%(7,2)
\rho_{72} &= 1 + \frac{32760 \nu ^4-190239 \nu ^3+273924 \nu^2-123489 \nu +16832} {14994 \left(7 \nu ^3-14 \nu ^2+7 \nu-1\right)} v_{\Omega }^2 \,, \\
%%(7,1)
\rho_{71} &= 1 + \frac{228 \nu ^3-2083 \nu ^2+2518 \nu -618}{714 \left(3 \nu ^2-4 \nu +1\right)} v_{\Omega }^2 \,, \\
\end{align*}

In [23]:
init_list.append(
"""        self.rho["(7 , 7)"] = (
            1 + 
            (
                (1380 * self.nu**3 - 4963 * self.nu**2 + 4246 * self.nu - 906) 
                / (714 * (3 * self.nu**2 - 4 * self.nu + 1))
            ) * self.vomega**2
            )
        
        self.rho["(7 , 6)"] = (
            1 + 
            (
                (6104 * self.nu**4 - 29351 * self.nu**3 + 37828 * self.nu**2 - 16185 * self.nu + 2144) 
                / (1666 * (7 * self.nu**3 - 14 * self.nu**2 + 7 * self.nu - 1))
            ) * self.vomega**2
            )
        
        self.rho["(7 , 5)"] = (
            1 + 
            (
                (804 * self.nu**3 - 3523 * self.nu**2 + 3382 * self.nu - 762) 
                / (714 * (3 * self.nu**2 - 4 * self.nu + 1))
            ) * self.vomega**2
            )
        
        self.rho["(7 , 4)"] = (
            1 + 
            (
                (41076 * self.nu**4 - 217959 * self.nu**3 + 298872 * self.nu**2 - 131805 * self.nu + 17756) 
                / (14994 * (7 * self.nu**3 - 14 * self.nu**2 + 7 * self.nu - 1))
            ) * self.vomega**2
            )
        
        self.rho["(7 , 3)"] = (
            1 + 
            (
                (420 * self.nu**3 - 2563 * self.nu**2 + 2806 * self.nu - 666) 
                / (714 * (3 * self.nu**2 - 4 * self.nu + 1))
            ) * self.vomega**2
            )
        
        self.rho["(7 , 2)"] = (
            1 + 
            (
                (32760 * self.nu**4 - 190239 * self.nu**3 + 273924 * self.nu**2 - 123489 * self.nu + 16832) 
                / (14994 * (7 * self.nu**3 - 14 * self.nu**2 + 7 * self.nu - 1))
            ) * self.vomega**2
            )
        
        self.rho["(7 , 1)"] = (
            1 + 
            (
                (228 * self.nu**3 - 2083 * self.nu**2 + 2518 * self.nu - 618) 
                / (714 * (3 * self.nu**2 - 4 * self.nu + 1))
            ) * self.vomega**2
            )
"""
)

The $l = 8$ modes are given in Equation B:

\begin{align*}
%%(8,8)
\rho_{88} &= 1+ \frac{3482 - 26778\nu + 64659\nu^2 - 53445\nu^3 + 12243\nu^4}{2736 (-1 + 7 \nu- 14 \nu^2 + 7 \nu^3)} v_\Omega^2\,, \\
%%(8,7)
\rho_{87} &= 1+ \frac{23478 - 154099\nu + 309498\nu^2 - 207550\nu^3 + 38920\nu^4}{18240 (-1 + 6\nu - 10\nu^2 + 4\nu^3)} v_\Omega^2\,, \\
%%(8,6)
\rho_{86} &= 1+ \frac{1002 - 7498\nu + 17269\nu^2 - 13055\nu^3 + 2653\nu^4}{912(-1 + 7\nu - 14\nu^2 + 7\nu^3)} v_\Omega^2\,, \\
%%(8,5)
\rho_{85} &= 1+ \frac{4350 - 28055\nu + 54642\nu^2 - 34598\nu^3 + 6056\nu^4}{3648 (-1 + 6\nu - 10\nu^2 + 4\nu^3)} v_\Omega^2\,, \\
%%(8,4)
\rho_{84} &= 1+ \frac{2666 - 19434\nu + 42627\nu^2 - 28965\nu^3 + 4899\nu^4}{2736(-1 + 7\nu - 14\nu^2 + 7\nu^3)} v_\Omega^2\,, \\
%%(8,3)
\rho_{83} &= 1+ \frac{20598 - 131059\nu + 249018\nu^2 - 149950\nu^3 + 24520\nu^4}{18240(-1 + 6\nu - 10\nu^2 + 4\nu^3)} v_\Omega^2\,, \\
%%(8,2)
\rho_{82} &= 1+ \frac{2462 - 17598\nu + 37119\nu^2 - 22845\nu^3 + 3063\nu^4}{2736(-1 + 7\nu - 14\nu^2 + 7\nu^3)} v_\Omega^2\,, \\
%%(8,1)
\rho_{81} &= 1+ \frac{20022 - 126451\nu + 236922\nu^2 - 138430\nu^3 + 21640\nu^4}{18240(-1 + 6\nu - 10\nu^2 + 4\nu^3)} v_\Omega^2\,.
\end{align*}

In [24]:
init_list.append(
"""        self.rho["(8 , 8)"] = 1 + ((3482 - 26778*self.nu + 64659*self.nu**2 - 53445*self.nu**3 + 12243*self.nu**4) /
                      (2736 * (-1 + 7*self.nu - 14*self.nu**2 + 7*self.nu**3))) * self.vomega**2
        
        self.rho["(8 , 7)"] = 1 + ((23478 - 154099*self.nu + 309498*self.nu**2 - 207550*self.nu**3 + 38920*self.nu**4) /
                      (18240 * (-1 + 6*self.nu - 10*self.nu**2 + 4*self.nu**3))) * self.vomega**2
        
        self.rho["(8 , 6)"] = 1 + ((1002 - 7498*self.nu + 17269*self.nu**2 - 13055*self.nu**3 + 2653*self.nu**4) /
                      (912 * (-1 + 7*self.nu - 14*self.nu**2 + 7*self.nu**3))) * self.vomega**2
        
        self.rho["(8 , 5)"] = 1 + ((4350 - 28055*self.nu + 54642*self.nu**2 - 34598*self.nu**3 + 6056*self.nu**4) /
                      (3648 * (-1 + 6*self.nu - 10*self.nu**2 + 4*self.nu**3))) * self.vomega**2
        
        self.rho["(8 , 4)"] = 1 + ((2666 - 19434*self.nu + 42627*self.nu**2 - 28965*self.nu**3 + 4899*self.nu**4) /
                      (2736 * (-1 + 7*self.nu - 14*self.nu**2 + 7*self.nu**3))) * self.vomega**2
        
        self.rho["(8 , 3)"] = 1 + ((20598 - 131059*self.nu + 249018*self.nu**2 - 149950*self.nu**3 + 24520*self.nu**4) /
                      (18240 * (-1 + 6*self.nu - 10*self.nu**2 + 4*self.nu**3))) * self.vomega**2
        
        self.rho["(8 , 2)"] = 1 + ((2462 - 17598*self.nu + 37119*self.nu**2 - 22845*self.nu**3 + 3063*self.nu**4) /
                      (2736 * (-1 + 7*self.nu - 14*self.nu**2 + 7*self.nu**3))) * self.vomega**2
        
        self.rho["(8 , 1)"] = 1 + ((20022 - 126451*self.nu + 236922*self.nu**2 - 138430*self.nu**3 + 21640*self.nu**4) /
                      (18240 * (-1 + 6*self.nu - 10*self.nu**2 + 4*self.nu**3))) * self.vomega**2
"""
)

<a id='fspin'></a>

# Step : $f^{\rm S}_{lm}$ \[Back to [top](#toc)\]
$$\label{fspin}$$

$f^{\rm S}_{lm}$ is defined in Appendix B of [PB2023](https://arxiv.org/pdf/2303.18039.pdf).

We begin with the (2,1) mode given in Equation B2b:

\begin{align*}
f^{\rm S}_{21} &= -\tfrac{3}{2} v_\Omega \left(\frac{\chi _A}{\delta} +\chi _S\right)
+ v_\Omega^3 \Big[\left(\tfrac{131}{84}\nu +\tfrac{61}{12}\right) \frac{\chi _A}{\delta} +\left(\tfrac{79}{84}\nu +\tfrac{61}{12}\right) \chi _S\Big]
+ v_\Omega^4 \left[(-2 \nu -3) \chi _A^2+ \left(\tfrac{21}{2}\nu -6\right) \frac{\chi _A \chi _S}{\delta}+\left(\tfrac{1}{2}\nu -3\right) \chi _S^2  \right] \nonumber\\
&\quad
+ v_\Omega^5 \Big[
\left(-\tfrac{703}{112}\nu^2 +\tfrac{8797}{1008}\nu -\tfrac{81}{16}\right) \frac{\chi _A}{\delta }+\left(\tfrac{613}{1008}\nu^2 +\tfrac{1709}{1008}\nu -\tfrac{81}{16}\right) \chi _S
+ \left(\tfrac{3}{4}-3 \nu \right) \frac{\chi _A^3}{\delta }+ \left(\tfrac{9}{4}-6 \nu \right) \frac{\chi _A \chi _S^2}{\delta }+\left(\tfrac{9}{4}-3 \nu \right) \chi _A^2 \chi _S+\tfrac{3}{4} \chi _S^3 \Big] \nonumber\\
&\quad
+ {v_\Omega^6 \Big[
\left(\tfrac{5}{7}\nu ^2 -\tfrac{9287}{1008}\nu +\tfrac{4163}{252}\right) \chi _A^2
+\left(\tfrac{139}{72} \nu ^2-\tfrac{2633 }{1008}\nu+\tfrac{4163}{252}\right) \chi _S^2
+ \left(\tfrac{9487}{504} \nu ^2 -\tfrac{1636}{21}\nu +\tfrac{4163}{126}\right) \frac{ \chi _A \chi _S}{\delta } \Big]},
\end{align*}

In the limiting case, this amounts to:

\begin{align*}
\lim_{\delta \rightarrow 0} \delta f^{\rm S}_{21} &= -\tfrac{3}{2} v_\Omega \left({\chi _A}\right)
+ v_\Omega^3 \Big[\left(\tfrac{131}{84}\nu +\tfrac{61}{12}\right) {\chi _A} \Big]
+ v_\Omega^4 \left[\left(\tfrac{21}{2}\nu -6\right){\chi _A \chi _S}\right] \nonumber\\
&\quad
+ v_\Omega^5 \Big[
\left(-\tfrac{703}{112}\nu^2 +\tfrac{8797}{1008}\nu -\tfrac{81}{16}\right) {\chi _A} + \left(\tfrac{3}{4}-3 \nu \right) {\chi _A^3}+ \left(\tfrac{9}{4}-6 \nu \right){\chi _A \chi _S^2}\Big] \nonumber\\
&\quad
+ {v_\Omega^6 \Big[\left(\tfrac{9487}{504} \nu ^2 -\tfrac{1636}{21}\nu +\tfrac{4163}{126}\right) \frac{ \chi _A \chi _S}{\delta } \Big]},
\end{align*}


In [25]:
init_list.append(
"""        self.fspin["(2 , 1)"] = (
            -sp.Rational(3, 2) * (
                self.chi_A * self.deltainv + self.chi_S
            ) * self.vomega
            + (
                (
                    sp.Rational(131, 84) * self.nu
                    + sp.Rational(61, 12)
                ) * (self.chi_A * self.deltainv)
                + (
                    sp.Rational(79, 84) * self.nu 
                    + sp.Rational(61, 12)
                ) * self.chi_S
            ) * self.vomega**3
            + (
                (-2 * self.nu - 3) * self.chi_A**2
                + (
                    sp.Rational(21, 2) * self.nu 
                    - 6
                ) * (self.chi_A * self.chi_S * self.deltainv)
                + (sp.Rational(1, 2) * self.nu - 3) * self.chi_S**2
            ) * self.vomega**4
            + (
                (
                    sp.Rational(-703, 112) * self.nu**2
                    + sp.Rational(8797, 1008) * self.nu
                    - sp.Rational(81, 16)
                ) * (self.chi_A * self.deltainv)
                + (
                    sp.Rational(613, 1008) * self.nu**2
                    + sp.Rational(1709, 1008) * self.nu
                    - sp.Rational(81, 16)
                ) * self.chi_S
                + (
                    sp.Rational(3, 4) 
                    - 3 * self.nu
                ) * (self.chi_A**3 * self.deltainv)
                + (
                    sp.Rational(9, 4) 
                    - 6 * self.nu
                ) * (self.chi_A * self.chi_S**2 * self.deltainv)
                + (sp.Rational(9, 4) - 3 * self.nu) * (self.chi_A**2 * self.chi_S)
                + sp.Rational(3, 4) * self.chi_S**3
            ) * self.vomega**5
            + (
                (
                    sp.Rational(5, 7) * self.nu**2 
                    - sp.Rational(9287, 1008) * self.nu 
                    + sp.Rational(4163, 252)
                ) * self.chi_A**2
                + (
                    sp.Rational(139, 72) * self.nu**2 
                    - sp.Rational(2633, 1008) * self.nu 
                    + sp.Rational(4163, 252)
                ) * self.chi_S**2
                + (
                    sp.Rational(9487, 504) * self.nu**2 
                    - sp.Rational(1636, 21) * self.nu 
                    + sp.Rational(4163, 126)
                ) * (self.chi_A * self.chi_S * self.deltainv)
            ) * self.vomega**6
            )
        self.fspin_limit["(2 , 1)"] = (
            -sp.Rational(3, 2) * (self.chi_A) * self.vomega
            + (
                (
                    sp.Rational(131, 84) * self.nu 
                    + sp.Rational(61, 12)
                ) * (self.chi_A)
            ) * self.vomega**2
            + (
                (sp.Rational(21, 2) * self.nu - 6) 
                * (self.chi_A * self.chi_S)
            ) * self.vomega**3
            + (
                (
                    sp.Rational(-703, 112) * self.nu**2
                    + sp.Rational(8797, 1008) * self.nu
                    - sp.Rational(81, 16)
                ) * (self.chi_A)
                + (
                    sp.Rational(3, 4) 
                    - 3 * self.nu
                ) * (self.chi_A**3)
                + (
                    sp.Rational(9, 4) 
                    - 6 * self.nu
                ) * (self.chi_A * self.chi_S**2)
            ) * self.vomega**5
            + (
                (
                    sp.Rational(9487, 504) * self.nu**2 
                    - sp.Rational(1636, 21) * self.nu 
                    + sp.Rational(4163, 126)
                ) * (self.chi_A * self.chi_S)
            ) * self.vomega**6
            )
"""
)

The (3,3) mode is given in Equation B3b:

\begin{align*}
f_{33}^\text{S}&= v_\Omega^3 \left[\left(\tfrac{19}{2}\nu-2\right) \frac{\chi _A}{\delta }+\left(\tfrac{5}{2}\nu-2\right) \chi _S\right] 
+ v_\Omega^4 \bigg[
\left(\tfrac{3}{2}-6 \nu \right) \chi _A^2+(3-12 \nu ) \frac{\chi _A \chi _S}{\delta }+\tfrac{3 }{2}\chi _S^2 \bigg] \nonumber \\
&\quad
+ v_\Omega^5 \left[\left(\tfrac{407}{30}\nu ^2-\tfrac{593}{60}\nu+\tfrac{2}{3}\right) \frac{ \chi _A}{\delta }+\left(\tfrac{241}{30}\nu ^2 +\tfrac{11}{20}\nu +\tfrac{2}{3}\right) \chi _S\right] \nonumber\\
&\quad
+ v_\Omega^6 \bigg[
\left(-12 \nu ^2+\tfrac{11}{2}\nu -\tfrac{7}{4}\right) \chi _A^2
+\left(44 \nu ^2-\nu -\tfrac{7}{2}\right) \frac{\chi _A \chi _S}{\delta }
+\left(6 \nu ^2-\tfrac{27}{2}\nu -\tfrac{7}{4}\right) \chi _S^2 
\bigg] \nonumber\\
&\quad
+ i \big(\Omega \mathcal{H}_{\rm real}\big)^2 \left[\left(\tfrac{7339}{540}\nu -\tfrac{81}{20}\right) \frac{\chi _A}{\delta }+\left(\tfrac{593}{108}\nu -\tfrac{81}{20}\right) \chi _S\right]
,\\
\end{align*}

In the limiting case:

\begin{align*}
\lim_{\delta \rightarrow 0} \delta f_{33}^\text{S}&= v_\Omega^3 \left[\left(\tfrac{19}{2}\nu-2\right) {\chi _A}\right] 
+ v_\Omega^4 \bigg[
\left(\tfrac{3}{2}-6 \nu \right) \chi _A^2+(3-12 \nu ) {\chi _A \chi _S} \bigg] \nonumber \\
&\quad
+ v_\Omega^5 \left[\left(\tfrac{407}{30}\nu ^2-\tfrac{593}{60}\nu+\tfrac{2}{3}\right){ \chi _A}\right] \nonumber\\
&\quad
+ v_\Omega^6 \bigg[
+\left(44 \nu ^2-\nu -\tfrac{7}{2}\right){\chi _A \chi _S}\bigg] \nonumber\\
&\quad
+ i \big(\Omega \mathcal{H}_{\rm real}\big)^2 \left[\left(\tfrac{7339}{540}\nu -\tfrac{81}{20}\right){\chi _A}\right]
,\\
\end{align*}

Note: In this case, $f^{\textrm{S}}_{33}$ needed for the amplitude is described by the real part only not the absolute value.

In [26]:
init_list.append(
"""        self.fspin["(3 , 3)"] = (
            (
                (sp.Rational(19, 2) * self.nu - 2) * self.chi_A * self.deltainv 
                + (sp.Rational(5, 2) * self.nu - 2) * self.chi_S
            ) * self.vomega**3
            + (
                (
                    sp.Rational(3, 2)
                    - 6 * self.nu
                ) * self.chi_A**2
                + (3 - 12 * self.nu) * (self.chi_A * self.chi_S * self.deltainv)
                + sp.Rational(3, 2) * self.chi_S**2
            ) * self.vomega**4
            + (
                (
                    sp.Rational(407, 30) * self.nu**2 
                    - sp.Rational(593, 60) * self.nu 
                    + sp.Rational(2, 3)
                ) * self.chi_A * self.deltainv
                + (
                    sp.Rational(241, 30) * self.nu**2 
                    + sp.Rational(11, 20) * self.nu 
                    + sp.Rational(2, 3)
                ) * self.chi_S
            ) * self.vomega**5
            + (
                (
                    -12 * self.nu**2 
                    + sp.Rational(11, 2) * self.nu 
                    - sp.Rational(7, 4)
                ) * self.chi_A**2
                + (
                    44 * self.nu**2 
                    - self.nu - sp.Rational(7, 2)
                ) * (self.chi_A * self.chi_S * self.deltainv)
                + (
                    6 * self.nu**2 
                    - sp.Rational(27, 2) * self.nu 
                    - sp.Rational(7, 4)) * self.chi_S**2
            ) * self.vomega**6
            )
        self.fspinimag = sp.I * (
            (
                sp.Rational(7339, 540) * self.nu 
                - sp.Rational(81, 20)
            ) * self.chi_A * self.deltainv 
            + (
                sp.Rational(593, 108) * self.nu 
                - sp.Rational(81, 20)
            ) * self.chi_S
            ) * self.vh3**2
        self.fspin_limit["(3 , 3)"] = (
            ( (sp.Rational(19, 2) * self.nu - 2) * self.chi_A) * self.vomega**3
            + ( (3 - 12 * self.nu) * (self.chi_A * self.chi_S) ) * self.vomega**4
            + (
                (
                    sp.Rational(407, 30) * self.nu**2 
                    - sp.Rational(593, 60) * self.nu 
                    + sp.Rational(2, 3)
                ) * self.chi_A
            ) * self.vomega**5
            + ( 
                (
                    44 * self.nu**2 
                    - self.nu - sp.Rational(7, 2)
                ) * (self.chi_A * self.chi_S)
            ) * self.vomega**6
            )
        self.fspinimag_limit = sp.I * self.chi_A * (
            sp.Rational(7339, 540) * self.nu 
            - sp.Rational(81, 20)
            ) * self.vh3**2
"""
)

The (3,1) mode is given in Equation B8:

\begin{align*}
f_{31}^\text{S} &= v_\Omega^3 \left[\left(\frac{11}{2}\nu -2\right) \frac{\chi _A}{\delta }+\left(\frac{13}{2}\nu-2\right) \chi _S\right], \\
\end{align*}

In the limiting case:

\begin{align*}
\lim_{\delta \rightarrow 0} \delta f_{31}^\text{S} &= v_\Omega^3 \left[\left(\frac{11}{2}\nu -2\right) {\chi _A}\right], \\
\end{align*}

In [27]:
init_list.append(
"""        self.fspin["(3 , 1)"] = (
            self.chi_A * self.deltainv * (-2 + sp.Rational(11,2) * self.nu) 
            + self.chi_S * (-2 + sp.Rational(13,2) * self.nu)
            ) * self.vomega**3
        self.fspin_limit["(3 , 1)"] = -self.chi_A * sp.Rational(5,8) * self.vomega**3
"""
)

The (4,3) mode is given in Equation B:

\begin{align*}
f_{43}^\text{S} &= \frac{v_\Omega}{1-2 \nu}\left(\tfrac{5}{2} \nu  \chi _S- \tfrac{5}{2}\nu\frac{\chi _A}{\delta}\right)
{+ \frac{v_\Omega^3}{1-2\nu} \left[
\left(\tfrac{887}{44}\nu -\tfrac{3143}{132} \nu ^2\right) \frac{\chi _A}{\delta}
+ \left(-\tfrac{529}{132} \nu ^2-\tfrac{667}{44}\nu\right) \chi _S
\right]}\nonumber\\
&\quad
+ {\frac{v_\Omega^4}{1-2\nu} \Big[
\left(12 \nu ^2-\tfrac{37}{3}\nu +\tfrac{3}{2}\right) \chi _A^2
+ \left(\tfrac{137}{6} \nu ^2-18 \nu +3\right) \frac{\chi _A \chi _S}{\delta}
+ \left(\tfrac{35}{6} \nu ^2+\tfrac{1 }{3}\nu+\tfrac{3}{2}\right) \chi _S^2
\Big]},\\
\end{align*}

In the limiting case:

\begin{align*}
\lim_{\delta \rightarrow 0} \delta f_{43}^\text{S} &= \frac{v_\Omega}{1-2 \nu}\left(- \tfrac{5}{2}\nu{\chi _A}\right)
{+ \frac{v_\Omega^3}{1-2\nu} \left[
\left(\tfrac{887}{44}\nu -\tfrac{3143}{132} \nu ^2\right) {\chi _A}\right]}\nonumber\\
&\quad
+ {\frac{v_\Omega^4}{1-2\nu} \Big[\left(\tfrac{137}{6} \nu ^2-18 \nu +3\right) {\chi _A \chi _S}
\Big]},\\
\end{align*}

In [28]:
init_list.append(
"""        self.fspin["(4 , 3)"] = (
            self.vomega / (1 - 2 * self.nu) * (sp.Rational(5, 2) * self.nu * self.chi_S - sp.Rational(5, 2) * self.nu * self.chi_A * self.deltainv)
            + self.vomega**3 / (1 - 2 *self.nu) * (
                (sp.Rational(887, 44) * self.nu - sp.Rational(3143, 132) * self.nu**2) * self.chi_A * self.deltainv
                + (-sp.Rational(529, 132) * self.nu**2 - sp.Rational(667, 44) * self.nu) * self.chi_S
            )
            + self.vomega**4 / (1 - 2 * self.nu) * (
                (12 * self.nu**2 - sp.Rational(37, 3) * self.nu + sp.Rational(3, 2)) * self.chi_A**2
                + (sp.Rational(137, 6) * self.nu**2 - 18 * self.nu + 3) * self.chi_A * self.chi_S * self.deltainv
                + (sp.Rational(35, 6) * self.nu**2 + sp.Rational(1, 3) * self.nu + sp.Rational(3, 2)) * self.chi_S**2
            )
        )
        
        self.fspin_limit["(4 , 3)"] = (
            self.vomega / (1 - 2 * self.nu) * (- sp.Rational(5, 2) * self.nu * self.chi_A)
            + self.vomega**3 / (1 - 2 * self.nu) * (
                (sp.Rational(887, 44) * self.nu - sp.Rational(3143, 132) * self.nu**2) * self.chi_A
            )
            + self.vomega**4 / (1 - 2 * self.nu) * (
                 (sp.Rational(137, 6) * self.nu**2 - 18 * self.nu + 3) * self.chi_A * self.chi_S
            )
        )
"""
)

The (4,1) mode is given in Equation B:

\begin{align*}
f_{41}^\text{S} &= \frac{5}{2} \nu  \frac{v_\Omega}{1-2 \nu } \left(\chi _S-\frac{\chi _A}{\delta }\right), \\
\end{align*}

In the limiting case:

\begin{align*}
\lim_{\delta \rightarrow 0} \delta f_{41}^\text{S} &= \frac{5}{2} \nu  \frac{v_\Omega}{1-2 \nu } \left(-{\chi _A}\right), \\
\end{align*}

In [29]:
init_list.append(
"""        self.fspin["(4 , 1)"] =(
            sp.Rational(5, 2) * self.nu * self.vomega / (1 - 2 * self.nu) * (self.chi_S - self.chi_A*self.deltainv)
        )
        self.fspin_limit["(4 , 1)"] =(
            sp.Rational(5, 2) * self.nu * self.vomega / (1 - 2 * self.nu) * (- self.chi_A)
        )
"""
)

The (5,5) mode is given in Equation B:

\begin{align*}
f_{55}^{\mathrm{S}} = & \left[\left(-\tfrac{70 \nu }{3 (-1+2 \nu )}+\tfrac{110 \nu ^2}{3 (-1+2 \nu )}+\tfrac{10}{3 (-1+2 \nu )}\right) \frac{\chi _A}{\delta}+\left(\tfrac{10}{3
   (-1+2 \nu )}-\tfrac{10 \nu }{-1+2 \nu }+\tfrac{10 \nu ^2}{-1+2 \nu }\right) \chi _S\right]v_\Omega^3 + \left[\tfrac{5}{2}\delta^2 \chi _A^2+5 \delta \chi _A \chi _S+\tfrac{5}{2}\chi _S^2\right]v_\Omega^4, \\
\end{align*}

In the limiting case:

\begin{align*}
\lim_{\delta \rightarrow 0} \delta f_{55}^{\mathrm{S}} = & \left[\left(-\tfrac{70 \nu }{3 (-1+2 \nu )}+\tfrac{110 \nu ^2}{3 (-1+2 \nu )}+\tfrac{10}{3 (-1+2 \nu )}\right) {\chi _A}\right]v_\Omega^3 , \\
\end{align*}

Note: Here, we implment pyseobnr's version of the $\mathcal{O}(v_{\Omega}^{4})$ which goes as follows:

$$
f_{55,\mathcal{O}(v_{\Omega}^{4})}^{\mathrm{S}} =
        \frac{5}{2}\chi_S^2 + \left(
            \frac{-5 }{ -1 + 2\nu} + \frac{30\nu}{-1 + 2\nu} - \frac{40\nu^2}{-1 + 2\nu}\right)\chi_A\chi_S\delta + \left(
            \frac{-5.0 }{ 2(-1 + 2\nu)} + \frac{15\nu}{-1 + 2\nu} - \frac{20\nu^2}{-1 + 2\nu} \right)\chi_A^2
$$

In [30]:
init_list.append(
"""        self.fspin["(5 , 5)"] = (
            (
                (
                    -70 * self.nu / (3*(-1 + 2 * self.nu))
                    + 110 * self.nu**2 / (3*(-1 + 2 * self.nu))
                    + 10 / (3*(-1 + 2 * self.nu))
                ) * (self.chi_A * self.deltainv)
                + (
                    10 / (3*(-1 + 2 * self.nu))
                    - 10 * self.nu / (-1 + 2 * self.nu)
                    + 10*self.nu**2 / (-1 + 2 * self.nu)
                ) * self.chi_S
            ) * self.vomega**3
            + (
                sp.Rational(5, 2) * self.chi_S**2
                + (
                    -5 / (-1 + 2 * self.nu)
                    + 30 * self.nu / (-1 + 2 * self.nu)
                    - 40 * self.nu**2 / (-1 + 2 * self.nu)
                ) * self.chi_A * self.chi_S * self.deltainv
                + (
                    -5 / (2 * (-1 + 2 * self.nu))
                    + 15 * self.nu / (-1 + 2 * self.nu)
                    - 20 * self.nu**2 / (-1 + 2 * self.nu)
                ) * self.chi_A**2
            ) * self.vomega**4
        )
        
        self.fspin_limit["(5 , 5)"] = (
            (
                (sp.Rational(-70, 3) * self.nu / ((-1 + 2 * self.nu))
                + sp.Rational(110, 3) * self.nu**2 / ((-1 + 2 * self.nu))
                + sp.Rational(10, 3) / ((-1 + 2 * self.nu))) * (self.chi_A)
            ) * self.vomega**3
            + (
                (
                    -5.0 / (-1.0 + 2.0 * self.nu)
                    + 30.0 * self.nu / (-1.0 + 2.0 * self.nu)
                    - 40.0 * self.nu**2 / (-1.0 + 2.0 * self.nu)
                ) * self.chi_A * self.chi_S    
            ) * self.vomega**4
        )
"""
)

<a id='deltalm'></a>

# Step : $\delta_{lm}$ \[Back to [top](#toc)\]
$$\label{deltalm}$$

$\delta_{lm}$ is defined in Appendix B of [PB2023](https://arxiv.org/pdf/2303.18039.pdf). We begin with the $l = 2$ modes:

\begin{align*}
\delta_{22} &=\tfrac{7 }{3}\Omega \mathcal{H}_{\rm real}
+ \big(\Omega \mathcal{H}_{\rm real}\big)^2 \left[\left(\tfrac{8}{3}\nu-\tfrac{4}{3}\right) \chi _S-\tfrac{4}{3}\delta \chi _A + \tfrac{428}{105} \pi\right] + \big(\Omega \mathcal{H}_{\rm real}\big)^3 \left[\tfrac{1712}{315} \pi^2-\tfrac{2203}{81} \right] - 24 \nu v_\Omega^5,\\
\delta_{21} &= \tfrac{2}{3}\Omega \mathcal{H}_{\rm real} +\tfrac{107}{105} \pi \big(\Omega \mathcal{H}_{\rm real}\big)^2 +\left(\tfrac{214}{315} \pi^2-\tfrac{272}{81}\right) \big(\Omega \mathcal{H}_{\rm real}\big)^3  - {\tfrac{25}{2}} \nu  v_\Omega^5,
\end{align*}

In [31]:
init_list.append(
"""        self.deltalm["(2 , 2)"] = (
            self.vh3 * sp.Rational(7, 3) 
            + self.vh3**2 * (
                (sp.Rational(8, 3) * self.nu - sp.Rational(4, 3)) * self.chi_S
                - sp.Rational(4, 3) * self.delta * self.chi_A
                + (sp.Rational(428, 105) * sp.pi)
            )
            + self.vh3**3 * (
                sp.Rational(1712, 315) * sp.pi**2
                - sp.Rational(2203, 81)
            )
            - self.vomega**5 * 24 * self.nu
            )
        self.deltalm["(2 , 1)"] = (
            self.vh3 * sp.Rational(2, 3)
            + self.vh3**2 * sp.Rational(107, 105) * sp.pi
            + self.vh3**3 * (
                sp.Rational(214, 315) * sp.pi**2
                - sp.Rational(272, 81)
            )
            - self.vomega**5 * sp.Rational(25, 2) * self.nu 
            )
"""
)

The $l = 3$ modes are :

\begin{align*}
\delta_{33} &=\tfrac{13}{10}\left(\mathcal{H}_{\mathrm{real}} \Omega\right)+\tfrac{39 \pi}{7}\left(\mathcal{H}_{\mathrm{real}} \Omega\right)^2+\left(-\tfrac{227827}{3000}+\tfrac{78 \pi^2}{7}\right)\left(\mathcal{H}_{\mathrm{real}} \Omega\right)^3-\tfrac{80897}{2430}\nu v_\Omega^5,\\
\delta_{32} &= \left(\tfrac{11}{5}\nu+\tfrac{2}{3}\right)  \frac{\Omega \mathcal{H}_\textrm{real}}{1-3 \nu } +\tfrac{52}{21} \pi(\Omega  \mathcal{H}_{\mathrm{real}})^2+\left(\tfrac{208}{63} \pi^2-\tfrac{9112}{405}\right) (\Omega  \mathcal{H}_{\mathrm{real}})^2,\\
\delta_{31} &= \frac{13}{30} \Omega  \mathcal{H}_{\mathrm{real}} + \frac{13}{21}\pi (\Omega  \mathcal{H}_{\mathrm{real}})^2 + \left(\frac{26}{63}\pi^2 - \frac{227827}{81000}\right)(\Omega  \mathcal{H}_{\mathrm{real}})^3, \\
\end{align*}

We only code up the two leading modes, (3,3) and (3,2), as those are the only modes implemented in SEOBNRv5HM. We state the other modes for completeness and future use.

In [32]:
init_list.append(
"""        self.deltalm["(3 , 3)"] = (
            sp.Rational(13, 10) * self.vh3 
            + self.vh3**2 * sp.Rational(39, 7) 
            + self.vh3**3 * (
                -sp.Rational(227827, 3000)
                + sp.Rational(78, 7) * sp.pi**2
            )
            + self.vomega**5 * (- sp.Rational(80897, 2430) * self.nu)
            )
        self.deltalm["(3 , 2)"] = (
            self.vh3 * (
                sp.Rational(11, 5) * self.nu 
                + sp.Rational(2, 3)
            ) / (1 - 3 * self.nu)
            + self.vh3**2 * sp.Rational(52, 21) * sp.pi
            + self.vh3**3 * (sp.Rational(208, 63) * sp.pi**2 - sp.Rational(9112, 405))
            )
"""
)

The $l = 4$ modes are:

\begin{align*}
\delta_{44} &= \tfrac{(112+219 \nu )}{120 (1-3 \nu )}(\Omega  \mathcal{H}_{\rm real})+\tfrac{25136 \pi}{3465}(\Omega  \mathcal{H}_{\rm real})^2 +\left(\tfrac{201088}{10395}\pi^2 - \tfrac{55144}{375} \right) (\Omega  \mathcal{H}_{\rm real})^3\,,\\
\delta_{43} &= \left(\tfrac{4961}{810}\nu+\tfrac{3}{5}\right) \frac{\Omega  H_\text{EOB}}{1-2 \nu } +\tfrac{1571}{385} \pi (\Omega  \mathcal{H}_{\rm real})^2,\\
\delta_{42} &= \left(\frac{7}{15} + \frac{14}{5}\nu\right) \frac{\Omega  \mathcal{H}_{\rm real}}{1-3\nu} + \frac{6284}{3465}\pi (\Omega  \mathcal{H}_{\rm real})^2,\\
\delta_{41} &= \left(\frac{1}{5}+\frac{507}{10}\nu\right)\frac{\Omega  \mathcal{H}_{\rm real}}{1-2\nu} + \frac{1571}{3465} \pi(\Omega  \mathcal{H}_{\rm real})^2,\\
\end{align*}

We only code up the two leading modes, (4,4) and (4,3), as those are the only modes implemented in SEOBNRv5HM. We state the other modes for completeness and future use.

In [33]:
init_list.append(
"""        self.deltalm["(4 , 4)"] = (
            self.vh3 * (
                sp.Rational(112, 120) 
                + sp.Rational(219, 120) * self.nu
            ) / (1 - 3* self.nu) 
            + self.vh3**2 * sp.Rational(25136, 3465) * sp.pi
            + self.vh3**3 * (sp.Rational(201088, 10395) * sp.pi**2 - sp.Rational(55144, 375))
            )
        self.deltalm["(4 , 3)"] = (
            self.vh3 * ( 
                sp.Rational(4961, 810) * self.nu 
                + sp.Rational(3, 5)
            ) / (1 - 2 * self.nu)
            + self.vh3**2 * sp.Rational(1571, 385) * sp.pi
            )
"""
)

The $l = 5$ modes are:

\begin{align*}
\delta_{55} =& \tfrac{(96875+857528 \nu )}{131250 (1-2 \nu )}(\Omega  H_{\mathrm{EOB}}) + \tfrac{3865\pi}{429}(\Omega  H_{\mathrm{EOB}})^2 + \tfrac{-7686949127 + 954500400\pi^2}{31783752}(\Omega  H_{\mathrm{EOB}})^3\,,\\
\delta_{54}&= \frac{8}{15} \Omega H_\text{EOB},\\
\delta_{53}&= \frac{31}{70} \Omega H_\text{EOB}, \\
\delta_{52}&= \frac{4}{15} \Omega H_\text{EOB}, \\
\delta_{51}&= \frac{31}{210} \Omega H_\text{EOB},
\end{align*}

We only code up the leading (5,5) mode as it is the only mode implemented in SEOBNRv5HM.

In [34]:
init_list.append(
"""        self.deltalm["(5 , 5)"] = (
            self.vh3 * (
                sp.Rational(96875, 131250) 
                + sp.Rational(857528, 131250) * self.nu
            ) / (1 - 2 * self.nu) 
            + self.vh3**2 * sp.Rational(3865, 429) * sp.pi
            + self.vh3**3 * (
                sp.Rational(-7686949127, 31783752)
                + sp.Rational(954500400, 31783752) * sp.pi**2
            )
            )
"""
)

# Step : The Dynamics Dependent Terms

<a id='khat'></a>

# Step : $\hat{k}$ \[Back to [top](#toc)\]
$$\label{khat}$$

The function is defined in Equation 47 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

\begin{equation*}
\hat{k} = m\Omega \mathcal{H}_{\rm real}
\end{equation*}

Where, $\Omega$, and $\mathcal{H}_{\rm real}$ are given as inputs

In [35]:
init_list.append(
"""        self.khat = []
        for m in range(9):
            self.khat.append(m * self.Omega * self.Hreal)
"""
)

<a id='eulerlog'></a>

# Step : ${\rm eulerlog}(m,v)$ \[Back to [top](#toc)\]
$$\label{eulerlog}$$

The function is defined in Equation 47 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

\begin{equation*}
{\rm eulerlog} (m,v) = \gamma_{E} + \log (2mv)
\end{equation*}

Where, $\gamma_E$ is the Euler-Mascheroni constant

In [36]:
init_list.append(
"""        eulerlog = [0,]
        for m in range(1,9):
            eulerlog.append(sp.EulerGamma + sp.log(2 * m * self.vomega))
"""
)

<a id='heff'></a>

# Step : $H_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{heff}$$

$H_{\rm eff}$ is defined immediately below Equation 32 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

\begin{equation*}
H_{\rm eff} = \frac{\mathcal{H}^{2}_{\rm real} - 1}{2\nu} + 1
\end{equation*}

Where, $\mathcal{H}_{\rm real}$ is the EOB Hamiltonian which is given as input.

In [37]:
init_list.append(
"""        Heff = (self.Hreal**2 - 1) / (2 * self.nu) + 1
"""
)

<a id='vH'></a>

# Step : $v_{H}$ \[Back to [top](#toc)\]
$$\label{vh}$$

$v_{H}$ is defined in the pyseobnr code as:

\begin{equation*}
v_{H}^3 = \Omega \mathcal{H}_{\rm real}
\end{equation*}

Where, $\Omega$ is the orbital frequency and $\mathcal{H}_{\rm real}$ is the real EOB Hamiltonian. Both are given as inputs.

In [38]:
init_list.append(
"""        self.vh3 = self.Omega * self.Hreal
"""
)

<a id='vphi'></a>

# Step : $v_{\phi}$ \[Back to [top](#toc)\]
$$\label{vphi}$$

$v_{\phi}$ is defined in Equation 30 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

\begin{equation*}
v_{\phi} = \Omega r_{\Omega}
\end{equation*}

Withe $r_{\Omega}$ defined in Equation 31 as:

\begin{equation*}
r_{\Omega} = \left.\left(\frac{\partial \mathcal{H}}{\partial p_{\phi}}\right)^{-2/3}\right|_{p_{r} = 0} = (\Omega_{\rm circ})^{-2/3}
\end{equation*}

Where, $\Omega$ is the orbital frequency and $\Omega_{circ}$ is the circular frequency. Both are given as inputs.

In [39]:
init_list.append(
"""        self.vphi = self.Omega * (self.Omega_circ**(-sp.Rational(2,3)))
"""
)

<a id='vomega'></a>

# Step : $v_{\Omega}$ \[Back to [top](#toc)\]
$$\label{vomega}$$

$v_{\Omega}$ is defined immediately above Equation 32 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

\begin{equation*}
v_{\Omega} = (\Omega)^{1/3}
\end{equation*}

Where, $\Omega$ is the orbital frequency which is given as input.

In [40]:
init_list.append(
"""        self.vomega = self.Omega**(sp.Rational(1,3))
"""
)

# Step : The Spin Terms

<a id='chi'></a>

# Step : ${\chi}_{A,S}$ \[Back to [top](#toc)\]
$$\label{chi}$$

$\chi_{A,S}$ is defined in Equation 3 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

\begin{equation*}
\chi_{A} = \frac{\chi_1 - \chi_2}{2}\\
\chi_{S} = \frac{\chi_1 + \chi_2}{2}
\end{equation*}

Where, $\chi_{1,2}$ are the primary and secondary dimensionless spin magnitudes respectively and are given as inputs.

In [41]:
init_list.append(
"""        self.chi_A = sp.Rational(1,2) * (chi1 - chi2)
        self.chi_S = sp.Rational(1,2) * (chi1 + chi2)
"""
)

# Step : The Mass Dependent Terms

<a id='delta'></a>

# Step : $\delta$ \[Back to [top](#toc)\]
$$\label{delta}$$

$\delta$ is defined in Equation 1 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

\begin{equation*}
\delta = \frac{m_1 - m_2}{M}
\end{equation*}

Where, $M$ is the total mass defined in [this cell](#M), and $m_{1,2}$ are the primary and secondary masses given as input.

As seen in the evaluation of the $f^{\rm S}_{lm}$, we need to be careful about calculating $1/\delta$ as the piecewise definition can still lead to NaN's. In order to avoid dividing by zero we define an invertible version of $\delta$ so that it returns the correct value when $\delta > 10^{-14}$ but does not diverge when $\delta < 10^{-14}$:

$$
\delta_{\rm invertible} = 
\bigg\lbrace
    \begin{array}{lr}
        \delta & \delta > 10^{-14},\\
        1 & \delta < 10^{-14},
    \end{array}\\
(\delta)^{-1} = 1/\delta_{\rm invertible}
$$

This ensures that there are no divergences. Additionally, the value of 1 is arbitrary as there is no $\delta$ dependence in $f^{\rm S}_{lm}$ when $\delta < 10^{-14}$.

Finally, the conditions for limiting equal mass behaviour are given as follows: (cf [NRPy+ Piecewise Expressions Tutorial](https://nbviewer.org/github/zachetienne/nrpytutorial/blob/master/Tutorial-Min_Max_and_Piecewise_Expressions.ipynb) for a full review on defining piecewise functions without `if` statements)

$$
(\delta < 1e-14) = \frac{\min(\delta - 10^{-14},0)}{\delta - 10^{-14} - {\rm TINYDOUBLE}}\\
(\delta \geq 1e-14) = \frac{\max(\delta - 10^{14} + {\rm TINYDOUBLE},0)}{\delta - 10^{-14} + {\rm TINYDOUBLE}}
$$

Where $TINYDOUBLE \equiv 1e-100$.

In [42]:
init_list.append(
"""        TINYDOUBLE = 1e-100
        self.delta = (m1 - m2) / M
        min_delta_1em14 = sp.Rational(1,2) * (self.delta - 1e-14 + 0 - sp.Abs(self.delta - 1e-14 - 0))
        max_delta_1em14 = sp.Rational(1,2) * (self.delta - 1e-14 + TINYDOUBLE + 0 + sp.Abs(self.delta - 1e-14 + TINYDOUBLE - 0))
        self.noneqcond = max_delta_1em14 / (self.delta - 1e-14 + TINYDOUBLE)
        self.eqcond = min_delta_1em14 /(self.delta - 1e-14 - TINYDOUBLE)
        self.deltainvertible = self.delta * self.noneqcond + 1 * self.eqcond
        self.deltainv = 1 / self.deltainvertible
"""
)

<a id='nu'></a>

# Step : $\nu$ \[Back to [top](#toc)\]
$$\label{nu}$$

$\nu$ is defined in Equation 1 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

\begin{equation*}
\nu = \frac{m_1 m_2}{M^2}
\end{equation*}

Where, $M$ is the total mass defined in [this cell](#m), and $m_{1,2}$ are the primary and secondary masses given as input.

In [43]:
init_list.append(
"""        self.nu = m1 * m2 / (M**2)
"""
)

<a id='m'></a>

# Step : $M$ \[Back to [top](#toc)\]
$$\label{m}$$

$M$ is defined in Equation 1 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

\begin{equation*}
M = {m_1 + m_2}
\end{equation*}

Where, $m_{1,2}$ are the primary and secondary masses given as input.

In [44]:
init_list.append(
"""        M = m1 + m2
"""
)

# Step : Saving the expressions

Up till now, the expressions required for the waveform have been stored in a .txt file. for the sake of readability, some of the expressions have been written in more than one line. 

Therefore, we save the expressions in one-line format to parse into sympy for generating optimized code.

In [49]:
import os 

module_header = """\"\"\"
Construct symbolic expression for the SEOBNRv5 aligned-spin gravitational-wave strain and flux.

Author: Siddharth Mahesh

License: BSD 2-Clause
\"\"\"


# Step P1: Import needed modules:
import sympy as sp

# The name of this module ("WaveEquation") is given by __name__:
thismodule = __name__


def f2r(input_float: float) -> sp.Rational:
    \"\"\"
    Convert a floating-point number to a high-precision rational number.

    This function takes a floating-point number, converts it to a string,
    and appends 60 zeros to increase the precision of the conversion to a rational number.

    :param input_float: The floating-point number to convert.
    :return: A sympy Rational number with high precision.

    >>> f2r(0.1)
    1/10
    >>> f2r(1.5)
    3/2
    >>> f2r(2.0)
    2
    \"\"\"
    # Convert the input float to a string
    float_as_string = str(input_float)

    # Ensure the string has a decimal point
    if "." not in float_as_string:
        float_as_string = f"{float_as_string}."

    # Append 60 zeros after the decimal of the floating point number to increase precision
    return sp.Rational(float_as_string + "0" * 60)


"""

class_def = """class SEOBNRv5_aligned_spin_waveform_quantities:
    \"\"\"Class for computing the SEOBNRv5 aligned-spin gravitational-wave strain and flux.\"\"\"
"""

init_def ="""
    def __init__(self) -> None:
        \"\"\"
        Compute the SEOBNRv5 aligned-spin Hamiltonian.

        This constructor sets up the necessary symbolic variables and expressions
        used in the computation of the aligned-spin SEOBNRv5 flux and strain. It
        initializes class variables like mass parameters, spin parameters, and
        various coefficients required for the waveforms's multipole modes.

        Inputs: 'm1', 'm2', 'r', 'phi', 'prstar', 'pphi', 'chi1', 'chi2', 'Hreal', 'Omega' and 'Omega_circ'
        Outputs: 'flux' and 'hlms'
        \"\"\"
        m1, m2, self.r, self.phi, self.prstar, self.pphi, chi1, chi2, self.Hreal, self.Omega, self.Omega_circ = sp.symbols(
            "m1 m2 r phi prstar pphi chi1 chi2 Hreal Omega Omega_circ", real=True
        )
        self.rho = {}
        for l in range(2,9):
            for m in range(1,l+1):
                self.rho.update({f"({l} , {m})" : 0})
        self.deltalm = {}
        self.fspin = {}
        self.fspin_limit = {}
        for l in range(2,6):
            for m in range(l,min(l-2,4),-1):
                self.deltalm.update({f"({l} , {m})" : 0})
                self.fspin.update({f"({l} , {m})" : 0})
                self.fspin_limit.update({f"({l} , {m})" : 0})
"""

flux_def = """
    def flux(self) -> sp.Basic:
        \"\"\"
        Compute the SEOBNRv5 aligned-spin flux.

        This function returns the symbolic expression for the factorized resummed flux
        in the SEOBNRv5 aligned spin approximant.

        Inputs: 'self'
        Returns:'flux'
        \"\"\"
        factorized_flux = 0
        for l in range(2 , 9):
            for m in range(1 , l+1):
"""

strain_def = """
    def strain(self) -> sp.Basic:
        \"\"\"
        Compute the SEOBNRv5 aligned-spin gravitational-wave strain.

        This function returns the symbolic expression for the factorized resummed strains
        in the SEOBNRv5 aligned spin approximant.

        Inputs: 'self'
        Returns: 'hlms'
        \"\"\"
        hlms = {}
        for l in range(2 , 6):
            for m in range(l , min(l-2,4), -1):
"""

validation_def = """
if __name__ == \"__main__\":
    import doctest
    import os
    import sys

    import nrpy.validate_expressions.validate_expressions as ve

    results = doctest.testmod()
    if results.failed > 0:
        print(f"Doctest failed: {results.failed} of {results.attempted} test(s)")
        sys.exit(1)
    else:
        print(f"Doctest passed: All {results.attempted} test(s) passed")

    results_dict = ve.process_dictionary_of_expressions(
        SEOBNRv5_aligned_spin_waveform_quantities().__dict__,
        fixed_mpfs_for_free_symbols=True,
    )
    ve.compare_or_generate_trusted_results(
        os.path.abspath(__file__),
        os.getcwd(),
        # File basename. If this is set to "trusted_module_test1", then
        #   trusted results_dict will be stored in tests/trusted_module_test1.py
        f"{os.path.splitext(os.path.basename(__file__))[0]}",
        results_dict,
    )
"""


with open(os.path.join(Ccodesdir,"SEOBNRv5_aligned_spin_waveform_quantities.py"), "w") as output:
    #write out the module header
    output.write(module_header)
    #define the Hamiltonian class
    output.write(class_def)
    #define the __init__ function
    output.write(init_def)
    # Read output of this notebook and populate it inside __init__
    for line in reversed(init_list):
        output.write(f"{line.replace('mpf(','f2r(').replace('.0 ',' ')}")
    #define the flux function
    output.write(flux_def)
    for line in reversed(flux_list):
        output.write(f"{line.replace('mpf(','f2r(').replace('.0 ',' ')}")
    output.write("        factorized_flux *= - (sp.Rational(1,8) * self.Omega / sp.pi)\n")
    output.write("        return factorized_flux\n")
    #define the strain function
    output.write(strain_def)
    for line in reversed(strain_list):
        output.write(f"{line.replace('mpf(','f2r(').replace('.0 ',' ')}")
    output.write("        return hlms\n")
    #write out the validation check
    output.write(validation_def)

!black Radiation/SEOBNRv5_aligned_spin_waveform_quantities.py
!pydocstyle Radiation/SEOBNRv5_aligned_spin_waveform_quantities.py
!pylint Radiation/SEOBNRv5_aligned_spin_waveform_quantities.py
!darglint Radiation/SEOBNRv5_aligned_spin_waveform_quantities.py

reformatted Radiation/SEOBNRv5_aligned_spin_waveform_quantities.py

All done! ✨ 🍰 ✨
1 file reformatted.
************* Module Radiation.SEOBNRv5_aligned_spin_waveform_quantities
Radiation/SEOBNRv5_aligned_spin_waveform_quantities.py:56:0: C0301: Line too long (107/100) (line-too-long)
Radiation/SEOBNRv5_aligned_spin_waveform_quantities.py:1:0: C0302: Too many lines in module (1457/1000) (too-many-lines)
Radiation/SEOBNRv5_aligned_spin_waveform_quantities.py:1:0: C0103: Module name "SEOBNRv5_aligned_spin_waveform_quantities" doesn't conform to snake_case naming style (invalid-name)
Radiation/SEOBNRv5_aligned_spin_waveform_quantities.py:13:0: C0103: Constant name "thismodule" doesn't conform to UPPER_CASE naming style (invalid-name)
Radiation/SEOBNRv5_aligned_spin_waveform_quantities.py:44:0: C0103: Class name "SEOBNRv5_aligned_spin_waveform_quantities" doesn't conform to PascalCase naming style (invalid-name)
Radiation/SEOBNRv5_aligned_spin_waveform_quantities.py:68:12: C0103: Attribute 

# Generate unoptimized python code

In this step, we'll simply store the waveform expressions in a python module as is in order to run preliminary validation tests.

# Generate optimized code

In [51]:
from Radiation.SEOBNRv5_aligned_spin_waveform_quantities import SEOBNRv5_aligned_spin_waveform_quantities
import sympy as sp
import nrpy.c_codegen as ccg

wf = SEOBNRv5_aligned_spin_waveform_quantities()
hlm, flux = wf.strain() , wf.flux()

print(type(flux))
print(type(hlm["(2 , 2)"]))
#hlm_ccode = ccg([hlm["(2 , 2)"] , hlm["(2 , 1)", hlm["(3 , 3)"] , hlm["(3 , 2)", hlm["(4 , 4)"] , hlm["(4 , 3)", hlm["(5 , 5)"]],['*h[2,2]' , '*h[2,1]' , '*h[3,3]' , '*h[3,2]' , '*h[4,4]' , '*h[4,3]' , '*h[5,5]'], verbose=False, include_braces=False)
h22_ccode = ccg.c_codegen(hlm["(2 , 2)"], "*h22", verbose = False, include_braces = False)
flux_ccode =ccg.c_codegen(flux , "*flux", verbose = False, include_braces = False)
with open(os.path.join(Ccodesdir,"v5HM_optimized_flux.py"),"w") as output:
    output.write('import numpy as np\nfrom scipy.special import gamma,factorial\n')
    output.write('import numpy as np\n')
    output.write("def v5HM_optimized_flux(m1,m2,r,phi,prstar,pphi,chi1,chi2,Hreal,Omega,Omega_circ):\n")
    output.write(flux_ccode.replace('const REAL ','    ')
                 .replace('const double ','    ')
                 .replace(';','')
                 .replace('*flux','    flux')
                 .replace("pow","np.power")
                 .replace("sqrt", "np.sqrt")
                 .replace("cbrt","np.cbrt")
                 .replace("log","np.log")
                 .replace("exp","np.exp")
                 .replace("fabs","np.absolute")
                 .replace("M_PI","np.pi")
                 .replace("I","1j")
                 .replace("tgamma","gamma")
                )
    output.write("    return flux")

with open(os.path.join(Ccodesdir,"v5HM_optimized_waveform.py"),"w") as output:
    output.write('import numpy as np\nfrom scipy.special import gamma,factorial\n')
    output.write("def v5HM_optimized_waveform(m1, m2, r, phi, prstar, pphi, chi1, chi2, Hreal, Omega, Omega_circ):\n")
    output.write(h22_ccode.replace('const REAL ','    ')
                 .replace('const double ','    ')
                 .replace(';','')
                 .replace('*h22','    h22')
                 .replace("pow","np.power")
                 .replace("sqrt", "np.sqrt")
                 .replace("cbrt","np.cbrt")
                 .replace("log","np.log")
                 .replace("exp","np.exp")
                 .replace("fabs","np.absolute")
                 .replace("M_PI","np.pi")
                 .replace("I","1j")
                 .replace("tgamma","gamma")
                )
    output.write("    return h22")

<class 'sympy.core.mul.Mul'>
<class 'sympy.core.mul.Mul'>


In [47]:
from Radiation.v5HM_optimized_flux import v5HM_optimized_flux as flux_nrpy
from Radiation.v5HM_unoptimized_flux import v5HM_unoptimized_flux as flux_unopt
from Radiation.pyseobnr_waveforms import compute_flux as flux_pyseobnr
import numpy as np

Validation = False
if Validation:
    gt_pert_total = []
    gt_pert_O1 = []
    gt_pert_O2 = []
    gt_pert_gtO3 = []
    rng = np.random.default_rng(seed = 42)
    nu = rng.random(1000000)*.25
    chi1 = 2.*rng.random(1000000)-1.
    chi2 = 2.*rng.random(1000000)-1.
    m2 = (1 - np.sqrt(1 - 4*nu))*.5
    m1 = (1 + np.sqrt(1 - 4*nu))*.5
    r = rng.random(1000000)*17. + 3.
    phi = rng.random(1000000)*2.*np.pi
    prstar = rng.random(1000000)*20. - 10.
    pphi = rng.random(1000000)*20. - 10.
    H = rng.random(1000000)
    Omega = rng.random(1000000)
    Omega_circ = rng.random(1000000)
    nans = []
    def E_rel(a,b):
        return np.abs((a - b)/a)

    max_erel = []
    for i in range(1000000):
        flux_ans = np.abs(flux_nrpy(m1[i], m2[i], r[i], phi[i], prstar[i], pphi[i], chi1[i], chi2[i],H[i],Omega[i],Omega_circ[i]))
        flux_unopt_value = flux_unopt(m1[i], m2[i], r[i], phi[i], prstar[i], pphi[i], chi1[i], chi2[i],H[i],Omega[i],Omega_circ[i])
        flux_true = flux_pyseobnr(m1[i], m2[i], r[i], phi[i], prstar[i], pphi[i], Omega[i], Omega_circ[i], H[i],chi1[i],chi2[i])/Omega[i]
        flux_pert = flux_pyseobnr(m1[i]*(1 + 1e-9), m2[i]*(1 + 1e-10), r[i], phi[i], prstar[i], pphi[i], Omega[i], Omega_circ[i], H[i],chi1[i],chi2[i])/Omega[i]
        e_rel = E_rel(flux_true,flux_ans)
        tol = E_rel(flux_true,flux_pert)
        if (np.isnan(flux_true) and np.isnan(flux_ans) and np.isnan(flux_pert)):
            nans.append(i)
        else:
            if e_rel>tol and tol > 0:
                print(f"case {i}: {flux_ans} , {flux_true} , {flux_pert}, erel : {e_rel}, tol : {tol}")
                gt_pert_total.append(i)
                if e_rel > 1000*tol:
                    if len(gt_pert_gtO3) == 0:
                        max_erel.append(i)
                        max_erel.append(e_rel)
                    elif e_rel > max_erel[1]:
                        max_erel[0] = i
                        max_erel[1] = e_rel
                    gt_pert_gtO3.append(i)
                elif e_rel > 100*tol:
                    gt_pert_O2.append(i)
                elif e_rel > 10*tol:
                    gt_pert_O1.append(i)

    print("Of total ",str(1000000)," comparisons,\n",
         "number of cases with relative error (total)  greater than allowed: ",len(gt_pert_total),"\n",
         "number of cases with relative error O(10)    greater than allowed: ",len(gt_pert_O1),"\n",
         "number of cases with relative error O(100)   greater than allowed: ",len(gt_pert_O2),"\n",
         "number of cases with relative error O(1000+) greater than allowed: ",len(gt_pert_gtO3))

In [48]:
flux_verbose_ccode = ccg.c_codegen([wf.flux(),wf.fspin["(2 , 1)"],wf.fspin["(3 , 3)"],wf.fspin["(3 , 1)"],wf.fspin["(4 , 3)"],wf.fspin["(4 , 1)"],wf.fspin["(5 , 5)"],wf.rho["(2 , 1)"],wf.rho["(2 , 2)"],wf.rho["(3 , 1)"],wf.rho["(3 , 2)"],wf.rho["(3 , 3)"],wf.rho["(4 , 1)"],wf.rho["(4 , 2)"],wf.rho["(4 , 3)"],wf.rho["(4 , 4)"],wf.rho["(5 , 1)"],wf.rho["(5 , 2)"],wf.rho["(5 , 3)"],wf.rho["(5 , 4)"],wf.rho["(5 , 5)"],wf.rho["(6 , 1)"],wf.rho["(6 , 2)"],wf.rho["(6 , 3)"],wf.rho["(6 , 4)"],wf.rho["(6 , 5)"],wf.rho["(6 , 6)"],wf.rho["(7 , 1)"],wf.rho["(7 , 2)"],wf.rho["(7 , 3)"],wf.rho["(7 , 4)"],wf.rho["(7 , 5)"],wf.rho["(7 , 6)"],wf.rho["(7 , 7)"],wf.rho["(8 , 1)"],wf.rho["(8 , 2)"],wf.rho["(8 , 3)"],wf.rho["(8 , 4)"],wf.rho["(8 , 5)"],wf.rho["(8 , 6)"],wf.rho["(8 , 7)"],wf.rho["(8 , 8)"],wf.effective_source[0],wf.effective_source[1],wf.Tlm["(2 , 2)"]],['*flux','fspin21','fspin33','fspin31','fspin43','fspin41','fspin55','rho21','rho22','rho31','rho32','rho33','rho41','rho42','rho43','rho44','rho51','rho52','rho53','rho54','rho55','rho61','rho62','rho63','rho64','rho65','rho66','rho71','rho72','rho73','rho74','rho75','rho76','rho77','rho81','rho82','rho83','rho84','rho85','rho86','rho87','rho88','source_even','source_odd','T22'],verbose = False, include_braces = False)

with open(os.path.join(Ccodesdir,"v5HM_optimized_flux_verbose.py"),"w") as output:
    output.write('import numpy as np\nfrom scipy.special import gamma,factorial\n')
    output.write('import numpy as np\n')
    output.write("def v5HM_optimized_flux_verbose(m1,m2,r,phi,prstar,pphi,chi1,chi2,Hreal,Omega,Omega_circ):\n")
    for line in flux_verbose_ccode.splitlines():
        output.write("    "+line.replace('const REAL ','')
                 .replace('const double ','')
                 .replace(';','')
                 .replace('*flux','flux')
                 .replace("pow","np.power")
                 .replace("sqrt", "np.sqrt")
                 .replace("cbrt","np.cbrt")
                 .replace("log","np.log")
                 .replace("exp","np.exp")
                 .replace("fabs","np.absolute")
                 .replace("M_PI","np.pi")
                 .replace("I","1j")
                 .replace("tgamma","gamma")
                 .replace("M_SQRT2","np.sqrt(2)")
                 +"\n"
                )
    output.write("    return flux , fspin21, fspin31, fspin33, fspin43, fspin41, fspin55, rho21, rho22, rho31, rho32, rho33, rho41, rho42, rho43, rho44, rho51, rho52, rho53, rho54, rho55, rho61, rho62, rho63, rho64, rho65, rho66, rho71, rho72, rho73, rho74, rho75, rho76, rho77, rho81, rho82, rho83, rho84, rho85, rho86, rho87, rho88, source_even, source_odd, T22")

from Radiation.v5HM_optimized_flux_verbose import v5HM_optimized_flux_verbose as flux_verbose
from Radiation.v5HM_unoptimized_flux import v5HM_unoptimized_flux as flux_unwf
flux , fspin21, fspin31, fspin33, fspin43, fspin41, fspin55, rho21, rho22, rho31, rho32, rho33, rho41, rho42, rho43, rho44, rho51, rho52, rho53, rho54, rho55, rho61, rho62, rho63, rho64, rho65, rho66, rho71, rho72, rho73, rho74, rho75, rho76, rho77, rho81, rho82, rho83, rho84, rho85, rho86, rho87, rho88, source_even, source_odd, T22 = flux_verbose(m1[i], m2[i], r[i], phi[i], prstar[i], pphi[i], chi1[i], chi2[i],H[i],Omega[i],Omega_circ[i])
flux_unopt_value, pnarray, rhoarray, fspinarray, Tarray, source_even_true, source_odd_true, delta, deltainv  = flux_unopt(m1[i], m2[i], r[i], phi[i], prstar[i], pphi[i], chi1[i], chi2[i],H[i],Omega[i],Omega_circ[i],verbose = True)



NameError: name 'm1' is not defined

In [ ]:
print(flux,flux_unopt_value)

In [ ]:
print(rho73, rhoarray[7][3])